# Processamento de linguagem natural para modelo de busca por voz.

# 1.0 Instalação

Para garantir o bom funcionamento do código de processamento de texto, análise de linguagem natural, vetorização, cálculo de similaridade e aprendizado de máquina para classificação, é imprescindível realizar as seguintes instalações e importações de bibliotecas. É de extrema importância que essas instalações estejam corretamente configuradas e que a máquina na qual o código será executado atenda aos requisitos necessários para seu funcionamento adequado.

## 1.1 Instalação da bibliotecas

In [3]:
!pip install unidecode
!pip install spacy
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.6 MB/s eta 0:00:00


## 1.2 Importação de bibliotecas

In [4]:
import pandas as pd
import numpy as np

from unidecode import unidecode

import spacy

#nltk para pre proscesamento e tokenização
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re

from nltk.stem import WordNetLemmatizer

nltk.download('rslp')

stemmer = nltk.stem.RSLPStemmer()

# Pacote nlp para portugues
nlp = spacy.cli.download('pt_core_news_sm')
nlp = spacy.load('pt_core_news_sm')

#Biblioteca usada para consultar uma URL
import requests
import urllib.request
from urllib.request import urlopen
from urllib.error import HTTPError

#Biblioteca usada analisar os dados retornados da url
from bs4 import BeautifulSoup


#importação de keras, para execução do modelo e criação do dicionario
from keras.preprocessing.text import Tokenizer

#setup w2vec
import gensim
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors

# Biblioteca para funções matemáticas
import math


# Google Drive
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
Mounted at /content/drive


# 2.0 Importação e compreensão dos dados

Nesta seção, realizamos a importação de dois dataframes. O dataframe é criado manualmente e possui informações semelhantes às que estão contidas no primeiro. Este dataframe é exclusivamente destinado aos testes de todas as funções que desenvolvemos ao longo deste notebook.

##2.1 DataFrame criado manualmente que se assemelha ao do banco de dados para realização de testes

In [5]:
df_url = pd.read_csv('/content/drive/Shareddrives/CogniVoice/ibm_db.csv')

In [6]:
df_url

,url_noticia
0,https://www.showmetech.com.br/ibm-mostra-avanc...
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...
2,https://forbes.com.br/forbes-tech/2023/07/pion...
3,https://inforchannel.com.br/2023/08/19/intelig...
4,https://itforum.com.br/ibm-quer-ampliar-base-w...
5,https://www.em.com.br/app/noticia/gerais/2023/...
6,https://epocanegocios.globo.com/futuro-do-dinh...
7,https://tecmasters.com.br/ceo-openai-diretora-...
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...
9,https://olhardigital.com.br/2023/08/22/ciencia...


In [7]:
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url_noticia  20 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


# 3.0 Tratamento dos dados

Nesta etapa, inicialmente criamos um dataframe manual contendo 12 frases aleatórias que servirão como entradas e saídas de teste para nossas funções. As funções que definimos nesta seção estão todas relacionadas ao processamento de texto, visando melhorar a compreensão por parte da máquina. É relevante destacar que, ao final desta seção, apresentamos como resposta os resultados de todas as funções aplicadas ao dataframe criado no início.

A seguir, listamos todas as funções:



*   Remoção de acentos
*   Tratamento de letras maiusculas
*   Tratamento do veiculo da noticia
*   Limpeza pontuação
*   Remoção de aspas
*   Remoção de espaços duplicados
*   Remoção de números isolados

Essas funções desempenham um papel fundamental na preparação e limpeza do texto, tornando-o mais adequado para análise posterior e garantindo um melhor desempenho dos modelos de aprendizado de máquina que serão aplicados.

In [8]:
dados_dataframe = [
    "Esta é a primeira frase.",
    "Aqui vem a segunda frase com palavras de parada.",
    "E a terceira frase.",
    "Quarta frase é necessário o uso de acentos e pontuações",
    "Teste para pontuação (virgulas, `""´ e dois pontos:)",
    "Exemplo de titulo 1  - Inforchannel",
    "Exemplo de titulo 2 | Voxel",
    "Exemplo de titulo 3 Por Investing.com",
    "  Isso  é um   exemplo   ",
    "  Espaços   múltiplos     ",
    "oi, tudo 3 bem 4 com voce?",
    "isso 76 apresenta 12 número isolados"
]

test_dataframe = pd.DataFrame(dados_dataframe)

test_dataframe

,0
0,Esta é a primeira frase.
1,Aqui vem a segunda frase com palavras de parada.
2,E a terceira frase.
3,Quarta frase é necessário o uso de acentos e p...
4,"Teste para pontuação (virgulas, `´ e dois pont..."
5,Exemplo de titulo 1 - Inforchannel
6,Exemplo de titulo 2 | Voxel
7,Exemplo de titulo 3 Por Investing.com
8,Isso é um exemplo
9,Espaços múltiplos


## 3.1 Remoção de acentos

Essa função é fundamental para melhorar o entendimento do texto pela máquina. Ela transforma todas as letras acentuadas em suas formas não acentuadas. Por exemplo, converte "é" em "e" e "nós" em "nos". Isso é importante porque simplifica o texto e evita que a presença de acentos atrapalhe o processamento.

### 3.1.1 Definição da função

In [9]:
def remocaoAcentos(text):
    return unidecode(text)

### 3.1.2 Teste da função

In [10]:
test_dataframe = test_dataframe.applymap(remocaoAcentos)

## 3.2 Tratamento de letras maiusculas

Transformar todas as letras em minúsculas é uma prática essencial para garantir a uniformidade do texto. Isso é necessário para que a máquina não diferencie letras maiúsculas de minúsculas, o que poderia prejudicar a análise do texto.

### 3.2.1 Definição da função

In [11]:
def maiusculas(dados):
    if isinstance(dados, pd.DataFrame):
        return dados.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    elif isinstance(dados, str):
        return dados.lower()
    else:
        return dados

def lowercase_text(text):
  return text.lower()

### 3.2.2 Teste da função

In [12]:
test_dataframe = test_dataframe.applymap(lowercase_text)

In [13]:
test_dataframe

,0
0,esta e a primeira frase.
1,aqui vem a segunda frase com palavras de parada.
2,e a terceira frase.
3,quarta frase e necessario o uso de acentos e p...
4,"teste para pontuacao (virgulas, `' e dois pont..."
5,exemplo de titulo 1 - inforchannel
6,exemplo de titulo 2 | voxel
7,exemplo de titulo 3 por investing.com
8,isso e um exemplo
9,espacos multiplos


## 3.3 Tratamento do veiculo da noticia

Usando expressões regulares (regex), esta função remove partes do título da notícia que não são relevantes. Por exemplo, em "IBM cria projeto com Inteli - Por Fulano", a parte " - Por Fulano" é excluída do título. Essa função ajuda a limpar o texto e é útil para filtrar notícias por título, se necessário.

### 3.3.1 Definição da função

In [14]:
def clean_title(match_comp):
    pattern = re.compile(r'^(.*?)\s*(-|\||por)\s*.*$')
    match = pattern.match(match_comp)
    return match.group(1) if match else match_comp

### 3.3.2 Teste da função

In [15]:
test_dataframe[0] = test_dataframe[0].apply(clean_title)

## 3.4 Limpeza pontuação

Esta função elimina todos os caracteres não alfanuméricos de uma string, como vírgulas, pontos de interrogação e exclamação. Isso é importante para o processamento de texto, pois tais caracteres podem aumentar o tempo de processamento e complicar a análise.

### 3.4.1 Definição da função

In [16]:
def remove_marks(text):
    if isinstance(text, str): #verifica se o que está sendo processado é uma string
        return re.sub(r'[^\w\s]|_', '', text) #substituir todos os caracteres especiais por uma string vazia
    else:
        return text

### 3.4.2 Teste da função

In [17]:
test_dataframe[0] = test_dataframe[0].apply(remove_marks)

## 3.5 Remoção de aspas

A remoção de aspas excessivas dentro de uma string é crucial para evitar problemas com modelos de vetorização que serão explicados posteriormente no código. Por exemplo, em "oi, tudo ''' bem?", as aspas repetidas são excluídas para evitar erros nos modelos.

### 3.5.1 Definição da função

In [18]:
def remove_aspas(column_or_text):
    if isinstance(column_or_text, pd.Series):  # Verifica se é uma coluna de DataFrame
        return column_or_text.apply(remove_aspas)
    elif isinstance(column_or_text, str):  # Verifica se é uma string
        return column_or_text.replace('"', '')
    else:
        return column_or_text  # Retorna outros tipos de dados sem modificação

### 3.5.2 Teste da função

In [19]:
test_dataframe = test_dataframe.apply(remove_aspas)

## 3.6 Remoção de espaços duplicados

Esta função elimina espaços em excesso dentro de uma string. Por exemplo, em "oi,      tudo bem?", os espaços duplicados são removidos. Isso é importante para evitar problemas com modelos de vetorização que serão explicados posteriormente no código.

### 3.6.1 Definição da função

In [20]:
def remove_duplicate_spaces(text):
    # Substitui sequências de espaços em branco por um único espaço
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

### 3.6.2 Teste da função

In [21]:
test_dataframe[0] = test_dataframe[0].apply(remove_duplicate_spaces)

## 3.7 Remoção de números isolados

Excluir números isolados de uma string, como em "oi, tudo 23 bem?", é importante para evitar erros nos modelos de vetorização que serão explicados mais adiante no código. A presença de números soltos pode complicar o processamento do texto.

### 3.7.1 Definição da função

In [22]:
def remove_isolated_numbers(text):
    # Substitui números isolados por um espaço em branco
    cleaned_text = re.sub(r'\b\d+\b', '', text)
    return cleaned_text.strip()

### 3.7.2 Definição da função

In [23]:
test_dataframe[0] = test_dataframe[0].apply(remove_isolated_numbers)

## 3.8 Dataframe resultante da seção

In [24]:
test_dataframe

,0
0,esta e a primeira frase
1,aqui vem a segunda frase com palavras de parada
2,e a terceira frase
3,quarta frase e necessario o uso de acentos e p...
4,teste para pontuacao virgulas e dois pontos
5,exemplo de titulo
6,exemplo de titulo
7,exemplo de titulo
8,isso e um exemplo
9,espacos multiplos


# 4.0 Web Scraping

Nesta seção, utilizaremos técnicas de Web Scraping para extrair e processar dados a partir da URL do nosso banco de dados. Como temos várias URLs, o processo consiste em fazer uma requisição ao site, obter o seu HTML e, em seguida, remover todas as tags que não serão utilizadas, como style e script, entre outras.

Além disso, nesta seção, realizaremos o tratamento para eliminar conteúdo não relevante na notícia, incluindo o conteúdo que aparece antes e depois da notícia, uma vez que coletamos todo o conteúdo da página HTML. É importante ressaltar que, ao final desta seção, apresentaremos o resultado em forma de um DataFrame.

## 4.1 Extração do titulo

Este código define uma função chamada "extrair_titulo" que recebe uma URL como entrada. Ela faz o seguinte:



1.   Define informações do navegador.
2.   Tenta acessar a URL da web usando essas informações.
3.   Extrai o título da página web.
4.   Retorna o título extraído ou uma string vazia em caso de erro.

Basicamente, a função recupera o título de uma página da web quando fornecida uma URL.

### 4.1.1 Definição da função

In [25]:
def extrair_titulo(url):
    print(url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }

    try:
        req = urllib.request.Request(url, headers=headers)
        page = urllib.request.urlopen(req)

        # response = requests.get(url, headers=headers)
        # response.raise_for_status()

        soup = BeautifulSoup(page, 'html.parser')
        title_new = soup.title.string
        title_new = clean_title(title_new)
        return title_new
    except HTTPError as e:
        print(f"Erro ao acessar a URL {url}: {e}")
        return ""  # Retorna um título vazio em caso de erro

### 4.1.2 Teste da função

In [27]:
df_url['titulo'] = df_url['url_noticia'].apply(extrair_titulo)

https://www.showmetech.com.br/ibm-mostra-avancos-da-watsonx/
https://inforchannel.com.br/2023/08/17/ibm-e-microsoft-expandem-parceria-para-oferecer-novas-solucoes-de-ia-generativa/
https://forbes.com.br/forbes-tech/2023/07/pioneira-em-ia-ibm-mostra-sua-estrategia-na-corrida-da-linguagem-generativa/
https://inforchannel.com.br/2023/08/19/inteligencia-artificial-da-ibm-comentara-jogos-no-torneio-us-open-de-tenis/
https://itforum.com.br/ibm-quer-ampliar-base-watsonx/
https://www.em.com.br/app/noticia/gerais/2023/09/07/interna_gerais,1558346/candidatos-de-minas-gerais-teriam-sido-excluidos-de-selecao-da-ibm.shtml
https://epocanegocios.globo.com/futuro-do-dinheiro/noticia/2023/08/bradesco-e-itau-se-aliam-a-ibm-para-testar-uso-da-computacao-quantica-no-mercado-financeiro.ghtml
https://tecmasters.com.br/ceo-openai-diretora-ibm-depoimento-senado/#:~:text=CEO%20da%20OpenAI%20e%20diretora%20da%20IBM%20prestam%20depoimento%20ao%20Senado,-Audi%C3%AAncia%20est%C3%A1%20marcada&text=Enquanto%20o%20Co

In [28]:
df_url['titulo']

0     IBM fala sobre avanços da WatsonX, plataforma ...
1     IBM e Microsoft expandem parceria para oferece...
2     Pioneira em IA, IBM mostra sua estratégia na c...
3     Inteligência Artificial da IBM comentará jogos...
4     IBM quer ampliar base de clientes Watson para ...
5     Candidatos de Minas Gerais teriam sido excluíd...
6     Bradesco e Itaú se aliam a IBM para testar uso...
7     CEO da OpenAI e diretora da IBM prestam depoim...
8       IBM lançará Llama 2 da Meta em plataforma de IA
9     IBM lança chip de IA analógico inspirado no cé...
10    Em acordo com a Meta, IBM vai disponilar Llama...
11               IBM Brasil tem novo líder de segurança
12    Projeto que utiliza inteligência artificial pr...
13    Com a NASA, IBM quer acelerar aplicações de IA...
14    IBM se aproxima de acordo de US$ 5 bilhões par...
15    Google, IBM, Microsoft e WhatsApp juntas no Su...
16    Valor Inovação: IBM Brasil se mantém como mais...
17                      IBM e Microsoft firmam p

## 4.2 Extração da noticia na integra e tratamento do dimensionamento

Este código define uma função chamada "remove_tags" que recebe uma URL como entrada. A função faz o seguinte:


1.   Chama a função "extrair_titulo" para obter o título da página da web a partir da URL.
2.   Realiza uma série de operações de processamento de texto no título obtido, incluindo remoção de acentos, conversão para letras maiúsculas e possivelmente uma função "clean_title".
3.   A função emula uma solicitação HTTP à URL usando informações de cabeçalho do navegador.
4.   Usa a biblioteca BeautifulSoup para analisar o conteúdo HTML da página web.
5.   Remove as tags HTML do conteúdo, incluindo tags de estilo, script e título.
6.   Extrai o texto remanescente da página, removendo espaços extras.
7.   Procura pelo título previamente processado dentro do texto e o recorta a partir desse ponto, se encontrado.
8.   Realiza mais operações de limpeza no texto, como remoção de pontuações, aspas, espaços duplicados e números isolados.
9.   Retorna o texto processado resultante.


Se ocorrer algum erro durante esse processo, a função imprime uma mensagem de erro e retorna uma string vazia. Em resumo, a função "remove_tags" extrai o texto de uma página da web, limpa-o e o retorna, com base no título da página.

### 4.2.1 Definição de função

In [124]:
def remove_tags(url):
    try:
        useless_tittle = extrair_titulo(url)
        useless_tittle = remocaoAcentos(useless_tittle)
        useless_tittle = maiusculas(useless_tittle)
        tittle = clean_title(useless_tittle)

        # parse html content
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
        }

        req = urllib.request.Request(url, headers=headers)
        page = urllib.request.urlopen(req)

        # page = requests.get(url, headers=headers)
        # page.raise_for_status()

        soup = BeautifulSoup(page, "html.parser")

        for data in soup(['style', 'script', 'title']):
            # Remove as tags
            data.decompose()

        # return data by retrieving the tag content
        all_text = ' '.join(soup.stripped_strings)

        all_text = maiusculas(all_text)
        all_text = remocaoAcentos(all_text)

        quebra = all_text.find(tittle)

        if quebra != -1:  # verifica encontrou a quebra
            all_text = all_text[quebra:]
            all_text = remove_marks(all_text)
            all_text = remove_aspas(all_text)
            all_text = remove_duplicate_spaces(all_text)
            all_text = remove_isolated_numbers(all_text)
            return all_text
        else:
            all_text = remove_marks(all_text)
            all_text = remove_aspas(all_text)
            all_text = remove_duplicate_spaces(all_text)
            all_text = remove_isolated_numbers(all_text)
            return all_text
    except Exception as e:
        print(f"Erro ao processar a URL {url}: {e}")
        return ""  # Retorna uma string vazia em caso de erro


In [125]:
remove_tags("https://www.google.com/")

https://www.google.com/


'google ai advertising business how search works our third decade of climate action join us privacy terms settings search settings advanced search your data in search search history search help send feedback dark theme off google apps'

### 4.2.2 Teste da função

In [30]:
df_url['tag_removed'] = df_url['url_noticia'].apply(remove_tags)

https://www.showmetech.com.br/ibm-mostra-avancos-da-watsonx/
https://inforchannel.com.br/2023/08/17/ibm-e-microsoft-expandem-parceria-para-oferecer-novas-solucoes-de-ia-generativa/
https://forbes.com.br/forbes-tech/2023/07/pioneira-em-ia-ibm-mostra-sua-estrategia-na-corrida-da-linguagem-generativa/
https://inforchannel.com.br/2023/08/19/inteligencia-artificial-da-ibm-comentara-jogos-no-torneio-us-open-de-tenis/
https://itforum.com.br/ibm-quer-ampliar-base-watsonx/
https://www.em.com.br/app/noticia/gerais/2023/09/07/interna_gerais,1558346/candidatos-de-minas-gerais-teriam-sido-excluidos-de-selecao-da-ibm.shtml
https://epocanegocios.globo.com/futuro-do-dinheiro/noticia/2023/08/bradesco-e-itau-se-aliam-a-ibm-para-testar-uso-da-computacao-quantica-no-mercado-financeiro.ghtml
https://tecmasters.com.br/ceo-openai-diretora-ibm-depoimento-senado/#:~:text=CEO%20da%20OpenAI%20e%20diretora%20da%20IBM%20prestam%20depoimento%20ao%20Senado,-Audi%C3%AAncia%20est%C3%A1%20marcada&text=Enquanto%20o%20Co

## 4.3 Dataframe resultante da seção

In [31]:
df_url

,url_noticia,titulo,tag_removed
0,https://www.showmetech.com.br/ibm-mostra-avanc...,"IBM fala sobre avanços da WatsonX, plataforma ...",ibm fala sobre avancos da watsonx plataforma d...
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...,IBM e Microsoft expandem parceria para oferece...,ibm e microsoft expandem parceria para oferece...
2,https://forbes.com.br/forbes-tech/2023/07/pion...,"Pioneira em IA, IBM mostra sua estratégia na c...",pioneira em ia ibm mostra sua estrategia na co...
3,https://inforchannel.com.br/2023/08/19/intelig...,Inteligência Artificial da IBM comentará jogos...,inteligencia artificial da ibm comentara jogos...
4,https://itforum.com.br/ibm-quer-ampliar-base-w...,IBM quer ampliar base de clientes Watson para ...,ibm quer ampliar base de clientes watson para ...
5,https://www.em.com.br/app/noticia/gerais/2023/...,Candidatos de Minas Gerais teriam sido excluíd...,candidatos de minas gerais teriam sido excluid...
6,https://epocanegocios.globo.com/futuro-do-dinh...,Bradesco e Itaú se aliam a IBM para testar uso...,bradesco e itau se aliam a ibm para testar uso...
7,https://tecmasters.com.br/ceo-openai-diretora-...,CEO da OpenAI e diretora da IBM prestam depoim...,ceo da openai e diretora da ibm prestam depoim...
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...,IBM lançará Llama 2 da Meta em plataforma de IA,ibm lancara llama da meta em plataforma de ia...
9,https://olhardigital.com.br/2023/08/22/ciencia...,IBM lança chip de IA analógico inspirado no cé...,ibm lanca chip de ia analogico inspirado no ce...


# 5.0 Extração dos dados

No processo de extração de dados, nosso principal objetivo é coletar informações adicionais que tornem mais simples para o usuário o filtro de notícias quando duas ou mais notícias correspondem no banco de dados.

A seguir, detalharemos quais informações estamos obtendo da notícia completa, conforme discutido na última seção sobre Web Scraping.



1.   Extração da Data: Este processo envolve a extração da data em que a notícia foi publicada.
2.   Extração da Fonte: Aqui, estamos coletando a fonte na qual a notícia foi originalmente veiculada.
3.   Extração de Entidades Nomeadas: Realizamos a extração de todas as organizações e pessoas mencionadas na notícia, a fim de utilizá-las como palavras-chave para facilitar as buscas dos usuários.

É relevante destacar que as duas primeiras funções utilizam expressões regulares (regex) para realizar a extração, enquanto a última emprega uma técnica de Processamento de Linguagem Natural (PLN) para realizar o mesmo procedimento. Os resultados dessas extrações podem ser encontrados ao final desta seção.

## 5.1 Extração da data

### 5.1.1 Definição da função

In [32]:
date_regex = [
  r'(\d{1,2} de [a-zA-Z]+ de \d{4})',
  r'(\d{1,2}/\d{1,2}/\d{2,4})',
  r'(\d{1,2} [a-zA-Z]+ \d{2,4})'
]

def extract_dates(row):
    quebra = row['tag_removed'].find(row['titulo'])
    for regex in date_regex:
        match = re.search(regex, row['tag_removed'][quebra:])
        if match:
            return match.group(1)
    return None

### 5.1.2 Teste da função

In [33]:
df_url['data'] = df_url.apply(extract_dates, axis=1)

## 5.2 Extração da fonte

### 5.2.1 Definição da função

In [34]:
def extrair_dominio(url):
    regex_url = r'https://(.*?)/'
    fonte = re.findall(regex_url, url)
    if fonte:
        return fonte[0]
    else:
        return None

### 5.2.2 Teste da função

In [35]:
df_url['fonte'] = df_url['url_noticia'].apply(extrair_dominio)

## 5.3 Extração das entidades nomeadas

### 5.3.1 Definição da função

In [36]:
def named_entities_filtred(coluna, filtro):
  results = []
  for linha in coluna:
    # print(linha)
    doc = nlp(linha)
    org_entities = set(ent.text for ent in doc.ents if ent.label_ == filtro)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    results.append(org_entities)
    # print("teste",results)
  return results

def named_entities(coluna):
  results = []
  for linha in coluna:
    # print(linha)
    doc = nlp(linha)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    results.append(entities)
    # print("teste",results)
  return results

### 5.3.2 Teste da função

In [37]:
teste_entities_df = pd.DataFrame()

df_url["empresas"] = named_entities_filtred(df_url['tag_removed'], "ORG")
df_url["pessoas"] = named_entities_filtred(df_url['tag_removed'], "PER")
teste_entities_df["Entidades"] = named_entities(df_url['tag_removed'])

## 5.4 Dataframe resultante da seção

In [38]:
df_url

,url_noticia,titulo,tag_removed,data,fonte,empresas,pessoas
0,https://www.showmetech.com.br/ibm-mostra-avanc...,"IBM fala sobre avanços da WatsonX, plataforma ...",ibm fala sobre avancos da watsonx plataforma d...,None,www.showmetech.com.br,"{hp, ibm institute for business value, bradesc...","{marcelo braga, victor pacheco, parquet avro, ..."
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...,IBM e Microsoft expandem parceria para oferece...,ibm e microsoft expandem parceria para oferece...,None,inforchannel.com.br,"{infor channel revista digital, ibm consulting...","{plus da manageengine zoho corporation, wwwibm..."
2,https://forbes.com.br/forbes-tech/2023/07/pion...,"Pioneira em IA, IBM mostra sua estratégia na c...",pioneira em ia ibm mostra sua estrategia na co...,None,forbes.com.br,"{openai, google, watsonx, ibm, nasa, ibm brasi...","{kasparov, watsonx, generativa ibm, bradesco v..."
3,https://inforchannel.com.br/2023/08/19/intelig...,Inteligência Artificial da IBM comentará jogos...,inteligencia artificial da ibm comentara jogos...,None,inforchannel.com.br,"{infor channel revista digital, ibm, infor cha...","{watsonx, jonathan adashek, red hat openshift,..."
4,https://itforum.com.br/ibm-quer-ampliar-base-w...,IBM quer ampliar base de clientes Watson para ...,ibm quer ampliar base de clientes watson para ...,None,itforum.com.br,"{watson, ibm institute for business value, ibm...","{watson, cio, watsonx, marcelo braga, platafor..."
5,https://www.em.com.br/app/noticia/gerais/2023/...,Candidatos de Minas Gerais teriam sido excluíd...,candidatos de minas gerais teriam sido excluid...,None,www.em.com.br,"{ibm foto, fale, uai tv, ibm, ibm brasil}","{paulo sindpdsp, lee, luciano huck bh dicas}"
6,https://epocanegocios.globo.com/futuro-do-dinh...,Bradesco e Itaú se aliam a IBM para testar uso...,bradesco e itau se aliam a ibm para testar uso...,None,epocanegocios.globo.com,"{bradesco, ibm quantum, ibm, google, ibm brasi...","{paris hilton, pipeline valor, gustavo pimenta..."
7,https://tecmasters.com.br/ceo-openai-diretora-...,CEO da OpenAI e diretora da IBM prestam depoim...,ceo da openai e diretora da ibm prestam depoim...,None,tecmasters.com.br,"{openai, google, ibm, pictwittercomvgynzebdg2}","{zac wolffunsplash, richard blumenthal, ted li..."
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...,IBM lançará Llama 2 da Meta em plataforma de IA,ibm lancara llama da meta em plataforma de ia...,None,forbes.com.br,"{openai, ibm watson, google, ibm, microsoft}","{mark zuckerberg luta, watsonx}"
9,https://olhardigital.com.br/2023/08/22/ciencia...,IBM lança chip de IA analógico inspirado no cé...,ibm lanca chip de ia analogico inspirado no ce...,None,olhardigital.com.br,"{ibm research, serie c0d3rs, ibm}",{ana luiza figueiredo}


# 6.0 Aplicação das tecnicas de PLN

Agora que concluímos o tratamento completo do texto, é hora de aplicar algumas técnicas de Processamento de Linguagem Natural (PLN) para facilitar a vetorização do texto e melhorar a compreensão por parte da máquina.

As técnicas de PLN que foram cuidadosamente escolhidas e aplicadas incluem:

1.   Stop Words
2.   Lematização
3.   Tokenização

É importante destacar que, durante nossa pesquisa e experimentação, consideramos a possibilidade de utilizar outras técnicas de PLN. No entanto, concluímos que muitas delas não proporcionavam melhorias substanciais no resultado final. Portanto, optamos por manter um conjunto mais simples e eficaz de técnicas de PLN para esta aplicação específica.

Ao final desta seção, você encontrará os resultados finais das aplicações dessas técnicas de PLN. Esses resultados demonstrarão como as etapas de pré-processamento melhoraram a qualidade dos dados e tornaram o texto mais adequado para análise por máquinas.

## 6.1 Stop Words

Eliminamos as palavras de parada, que são palavras muito comuns na língua e geralmente não contribuem significativamente para o entendimento do texto. Essa etapa ajuda a reduzir o ruído nos dados e a melhorar o desempenho dos modelos de PLN.

### 6.1.1 Definição da função

In [39]:
def stopWords(dados):
    if isinstance(dados, str):
        dados = [dados]

    resultado = [' '.join([token.text for token in nlp(texto) if not token.is_stop]) for texto in dados]

    return resultado

### 6.1.2 Teste da função

In [40]:
stopWords("um projeto realizado em parceria entre nasa e ibm pretende reduzir [...] em tempo quase que real")

['projeto realizado parceria nasa ibm pretende reduzir [ ... ] quase real']

In [41]:
df_url['tag_removed'] = stopWords(df_url['tag_removed'])

## 6.2 Lematização

Aplicamos a lematização para reduzir as palavras às suas formas básicas ou lemas. Isso ajuda a consolidar palavras relacionadas e simplificar a análise, já que diferentes formas de uma palavra são tratadas como uma única unidade.

### 6.2.1 Definição da função

In [42]:
def lematizar(dados):
    resultado = []
    for texto in dados:
        # Processa o texto com o modelo do Spacy
        doc = nlp(texto)
        # Lematiza o documento
        lemmas = [token.lemma_ for token in doc]
        # Junta tudo de volta a um texto
        textoLemmatizado = ' '.join(lemmas)
        resultado.append(textoLemmatizado)
    return resultado

### 6.2.2 Teste da função

In [43]:
teste_string = [
    "projeto realizado parceria nasa ibm pretende reduzir tempo quase real"
]

lema_dataframe = pd.DataFrame(teste_string)

lema_dataframe[0] = lematizar(lema_dataframe[0])

In [44]:
lema_dataframe

,0
0,projeto realizar parceria nasa ibm pretender r...


In [45]:
df_url['tag_removed'] = lematizar(df_url['tag_removed'])

## 6.3 Tokenização

Dividimos o texto em tokens, que são unidades individuais, como palavras ou partes de palavras. Isso é fundamental para que a máquina compreenda a estrutura do texto e possa processá-lo de maneira eficaz.

### 6.3.1 Definição da função

In [46]:
def tokenizar_texto(texto):
    doc = nlp(texto)
    tokens = [token.text for token in doc]
    return tokens

### 6.3.2 Teste da função

In [47]:
token_string = tokenizar_texto("projeto realizar parceria nasa ibm pretender reduzir tempo quase real")
token_string

['projeto',
 'realizar',
 'parceria',
 'nasa',
 'ibm',
 'pretender',
 'reduzir',
 'tempo',
 'quase',
 'real']

In [48]:
df_url['tag_removed'] = df_url['tag_removed'].apply(tokenizar_texto)

## 6.4 Dataframe resultante da seção

In [49]:
df_url

,url_noticia,titulo,tag_removed,data,fonte,empresas,pessoas
0,https://www.showmetech.com.br/ibm-mostra-avanc...,"IBM fala sobre avanços da WatsonX, plataforma ...","[ibm, falar, avanco, Watsonx, plataformar, ai,...",None,www.showmetech.com.br,"{hp, ibm institute for business value, bradesc...","{marcelo braga, victor pacheco, parquet avro, ..."
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...,IBM e Microsoft expandem parceria para oferece...,"[ibm, microsoft, expander, parceria, oferecer,...",None,inforchannel.com.br,"{infor channel revista digital, ibm consulting...","{plus da manageengine zoho corporation, wwwibm..."
2,https://forbes.com.br/forbes-tech/2023/07/pion...,"Pioneira em IA, IBM mostra sua estratégia na c...","[pioneirar, ir, ibm, mostrar, estrategio, corr...",None,forbes.com.br,"{openai, google, watsonx, ibm, nasa, ibm brasi...","{kasparov, watsonx, generativa ibm, bradesco v..."
3,https://inforchannel.com.br/2023/08/19/intelig...,Inteligência Artificial da IBM comentará jogos...,"[inteligencia, artificial, ibm, comentar, jogo...",None,inforchannel.com.br,"{infor channel revista digital, ibm, infor cha...","{watsonx, jonathan adashek, red hat openshift,..."
4,https://itforum.com.br/ibm-quer-ampliar-base-w...,IBM quer ampliar base de clientes Watson para ...,"[ibm, ampliar, base, cliente, watson, Watsonx,...",None,itforum.com.br,"{watson, ibm institute for business value, ibm...","{watson, cio, watsonx, marcelo braga, platafor..."
5,https://www.em.com.br/app/noticia/gerais/2023/...,Candidatos de Minas Gerais teriam sido excluíd...,"[candidato, mina, geral, ter, ser, excluir, se...",None,www.em.com.br,"{ibm foto, fale, uai tv, ibm, ibm brasil}","{paulo sindpdsp, lee, luciano huck bh dicas}"
6,https://epocanegocios.globo.com/futuro-do-dinh...,Bradesco e Itaú se aliam a IBM para testar uso...,"[bradesco, itaur, al, ibm, testar, uso, comput...",None,epocanegocios.globo.com,"{bradesco, ibm quantum, ibm, google, ibm brasi...","{paris hilton, pipeline valor, gustavo pimenta..."
7,https://tecmasters.com.br/ceo-openai-diretora-...,CEO da OpenAI e diretora da IBM prestam depoim...,"[ceo, openai, diretora, ibm, prestam, depoimen...",None,tecmasters.com.br,"{openai, google, ibm, pictwittercomvgynzebdg2}","{zac wolffunsplash, richard blumenthal, ted li..."
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...,IBM lançará Llama 2 da Meta em plataforma de IA,"[ibm, lancar, llama, , meta, plataforma, ir...",None,forbes.com.br,"{openai, ibm watson, google, ibm, microsoft}","{mark zuckerberg luta, watsonx}"
9,https://olhardigital.com.br/2023/08/22/ciencia...,IBM lança chip de IA analógico inspirado no cé...,"[ibm, lanca, chip, ir, analogico, inspirar, ce...",None,olhardigital.com.br,"{ibm research, serie c0d3rs, ibm}",{ana luiza figueiredo}


In [50]:
print(df_url["titulo"][0])
print(df_url["titulo"][0])

IBM fala sobre avanços da WatsonX, plataforma de AI para empresas
IBM fala sobre avanços da WatsonX, plataforma de AI para empresas


# 7.0 Pipeline de pré processamento

Neste estágio, nosso objetivo é consolidar todas as funções anteriormente desenvolvidas em uma única e acessível pipeline. Com esse propósito, criamos duas pipelines de dados distintas. A primeira é responsável por processar o DataFrame contendo as URLs, enquanto a segunda lida com o tratamento de uma entrada de texto em formato de string, que representa os dados fornecidos pelo usuário por meio do backend.

1.   Pipeline de Tratamento de DataFrame:
2.   Pipeline de Tratamento de Entrada do Usuário:

Ambas as pipelines foram criadas para simplificar e otimizar a maneira como tratamos dados brutos e entradas de texto do usuário, garantindo que nossas análises e pesquisas sejam eficazes e precisas. Elas representam uma abordagem eficiente para o processamento de informações em diferentes contextos e constituem uma parte fundamental de nossa estrutura de processamento de dados.

## 7.1 Importação de um novo dataframe

In [51]:
df_pipeline = pd.read_csv('/content/drive/Shareddrives/CogniVoice/ibm_db.csv')

In [52]:
df_pipeline

,url_noticia
0,https://www.showmetech.com.br/ibm-mostra-avanc...
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...
2,https://forbes.com.br/forbes-tech/2023/07/pion...
3,https://inforchannel.com.br/2023/08/19/intelig...
4,https://itforum.com.br/ibm-quer-ampliar-base-w...
5,https://www.em.com.br/app/noticia/gerais/2023/...
6,https://epocanegocios.globo.com/futuro-do-dinh...
7,https://tecmasters.com.br/ceo-openai-diretora-...
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...
9,https://olhardigital.com.br/2023/08/22/ciencia...


## 7.2 Definição da função pipeline para dataframe

Nossa primeira pipeline foi projetada para realizar um conjunto abrangente de etapas de processamento de dados no DataFrame que armazena as URLs. Isso inclui a extração, limpeza e transformação dos dados brutos em um formato adequado para análise e pesquisa. Essa pipeline torna a manipulação de URLs e a extração de informações relevantes um processo eficiente e organizado.

In [53]:
def pipeline_dataframe(dados, colum_name):
  #Cria um dataframe vazio para armazenar os dados tratados
  treated_data =  pd.DataFrame()

  #Busca pela url o site, faz o tratamento do html e retorna o texto da noticia
  treated_data["data"] = dados[colum_name].apply(remove_tags)

  treated_data["data"] = treated_data["data"].apply(remove_duplicate_spaces)

  #Remove possiveis acentos da tabela
  treated_data = treated_data.applymap(remocaoAcentos)

  treated_data = treated_data.applymap(lowercase_text)

  treated_data["data"] = stopWords(treated_data["data"])

  treated_data["data"] = lematizar(treated_data["data"])

  treated_data["data"] = treated_data["data"].apply(tokenizar_texto)

  return treated_data


## 7.3 Definição da função pipeline entrada do usuario

A segunda pipeline foi desenvolvida para lidar com as informações fornecidas pelo usuário por meio do backend na forma de uma string. Essa entrada do usuário passa por um conjunto de processos que a preparam para análise, incluindo a aplicação das técnicas de Processamento de Linguagem Natural (PLN) mencionadas anteriormente, como a remoção de palavras de parada, lematização e tokenização. Dessa forma, garantimos que o texto do usuário esteja em uma forma adequada para compreensão e pesquisa.

In [54]:
def pipeline_speech_to_text(text):
  array_phrase = []

  text = remocaoAcentos(text)

  text = lowercase_text(text)

  text = remove_marks(text)

  text = stopWords(text)

  text = lematizar(text)

  text = tokenizar_texto(str(text)[1:-1].strip("'"))

  array_phrase.append(text)

  return array_phrase

## 7.3 Teste da função

In [55]:
teste_pipeline = pipeline_dataframe(df_pipeline, "url_noticia")

https://www.showmetech.com.br/ibm-mostra-avancos-da-watsonx/
https://inforchannel.com.br/2023/08/17/ibm-e-microsoft-expandem-parceria-para-oferecer-novas-solucoes-de-ia-generativa/
https://forbes.com.br/forbes-tech/2023/07/pioneira-em-ia-ibm-mostra-sua-estrategia-na-corrida-da-linguagem-generativa/
https://inforchannel.com.br/2023/08/19/inteligencia-artificial-da-ibm-comentara-jogos-no-torneio-us-open-de-tenis/
https://itforum.com.br/ibm-quer-ampliar-base-watsonx/
https://www.em.com.br/app/noticia/gerais/2023/09/07/interna_gerais,1558346/candidatos-de-minas-gerais-teriam-sido-excluidos-de-selecao-da-ibm.shtml
https://epocanegocios.globo.com/futuro-do-dinheiro/noticia/2023/08/bradesco-e-itau-se-aliam-a-ibm-para-testar-uso-da-computacao-quantica-no-mercado-financeiro.ghtml
https://tecmasters.com.br/ceo-openai-diretora-ibm-depoimento-senado/#:~:text=CEO%20da%20OpenAI%20e%20diretora%20da%20IBM%20prestam%20depoimento%20ao%20Senado,-Audi%C3%AAncia%20est%C3%A1%20marcada&text=Enquanto%20o%20Co

In [56]:
phrase_teste = "Bom dia, existe algum projeto da USP em parceria com a IBM?"

phrase_teste = pipeline_speech_to_text(phrase_teste)

phrase_teste

[['dia', 'existir', 'algum', 'projeto', 'usp', 'parcerio', 'ibm']]

## 7.4 Dataframe resultante da seção

In [57]:
teste_pipeline

,data
0,"[ibm, falar, avanco, Watsonx, plataformar, ai,..."
1,"[ibm, microsoft, expander, parceria, oferecer,..."
2,"[pioneirar, ir, ibm, mostrar, estrategio, corr..."
3,"[inteligencia, artificial, ibm, comentar, jogo..."
4,"[ibm, ampliar, base, cliente, watson, Watsonx,..."
5,"[candidato, mina, geral, ter, ser, excluir, se..."
6,"[bradesco, itaur, al, ibm, testar, uso, comput..."
7,"[ceo, openai, diretora, ibm, prestam, depoimen..."
8,"[ibm, lancar, llama, meta, plataformo, ir, emp..."
9,"[ibm, lanca, chip, ir, analogico, inspirar, ce..."


# 8.0 Bag of Words

O BoW é uma técnica simples que converte um documento de texto em um vetor contendo a contagem de ocorrências de cada palavra no documento. Cada palavra é tratada como uma característica independente, ignorando a ordem e a estrutura das palavras no texto.

## 8.1 Definição das funções isoladas

In [58]:
# função para vetor de recorrencia

def bagOfWords(dicionario,dados):
  salvar = dicionario.texts_to_matrix(dados, mode='count')
  return salvar

In [59]:
def criar_dicionario(dados):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(dados)
  return tokenizer

## 8.2 Teste individual das funções

In [60]:
teste_bow = [
    "projeto,realizar,parceria,nasa,ibm,pretender,reduzir,tempo,quase,real",
    "projeto,realizar,parceria,microsoft,ibm,inteligencia,artificial,generativa,nuvem"
]

bow_dataframe = pd.DataFrame(teste_bow)

In [61]:
dicionarioTeste = criar_dicionario(bow_dataframe[0])
print(dicionarioTeste)
print(list(dicionarioTeste.word_index.keys()))
print(len(list(dicionarioTeste.word_index.keys())))

['projeto', 'realizar', 'parceria', 'ibm', 'nasa', 'pretender', 'reduzir', 'tempo', 'quase', 'real', 'microsoft', 'inteligencia', 'artificial', 'generativa', 'nuvem']
15


In [62]:
dicionarioTeste2 = criar_dicionario(teste_pipeline["data"])
print(dicionarioTeste2)
print(list(dicionarioTeste2.word_index.keys()))
print(len(list(dicionarioTeste2.word_index.keys())))

['ibm', 'ir', 'empresa', 'tecnologia', 'brasil', 'modelo', 'inteligencia', 'artificial', 'tambem', 'ser', 'generativo', 'ano', 'usp', 'dar', 'ha', 'watsonx', 'nao', 'uso', 'quantico', 'ajudar', 'digital', 'dado', 'ja', 'negocio', 'computacao', 'noticio', 'cliente', 'sao', 'ai', 'politico', 'estao', 'trabalho', 'brasileiro', 'estar', 'hora', 'falar', 'mercado', 'ate', 'aler', 'privacidade', 'computador', 'pesquisa', 'consulting', 'anunciar', 'email', 'setor', 'projeto', 'lider', 'r', 'microsoft', 'criar', 'paulo', 'economia', 'direito', 'us', 'caso', 'incluir', 'globo', 'mundo', 'analise', 'ceo', 'servico', 'indigena', 'processo', 'foto', 'de', 'linguagem', 'abrir', 'conteudo', 'trabalhar', 'voce', 'watson', 'apresentar', 'azure', 'c', 'chip', 'mostrar', 'bradesco', 'plataforma', 'dia', 'google', 'experiencia', 'evento', 'possivel', 'fazer', 'comentario', 'newsletter', 'reservar', 'openai', 'financeiro', 'banco', 'adobe', 'desenvolver', 'ver', 'bom', 'conjunto', 'principal', 'executivo'

In [63]:
bagOfWords(dicionarioTeste,bow_dataframe[0])

array([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]])

In [64]:
bagOfWords(dicionarioTeste2,teste_pipeline["data"])

array([[ 0., 20.,  9., ...,  0.,  0.,  0.],
       [ 0., 18., 22., ...,  0.,  0.,  0.],
       [ 0., 13., 20., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  9.,  4., ...,  0.,  0.,  0.],
       [ 0., 19., 17., ...,  0.,  0.,  0.],
       [ 0.,  3.,  2., ...,  1.,  1.,  1.]])

## 8.3 Definição e criação do modelo

In [65]:
#função de vetorização de Bag of words
def Bow(dados):
  #Recebe os dados a ser realizado o BOW
  dadosTratados = dados
  dadosTratados =  pd.DataFrame({'dadosBow': dadosTratados})
  #geração de dicionario
  dicionario = criar_dicionario(dadosTratados['dadosBow'])
  # chamar o modelo de bag of words
  modelo = (bagOfWords(dicionario,dadosTratados['dadosBow']))
  return modelo,dicionario

#função de vetorização de Bag of words
def Bow_dic(dados, dicionario):
  #Recebe os dados a ser realizado o BOW
  # dadosTratados = dados
  # dadosTratados =  pd.DataFrame({'dadosBow': dadosTratados})
  # chamar o modelo de bag of words
  modelo = (bagOfWords(dicionario,dados[0]))
  return modelo,dicionario

## 8.4 Teste do modelo BoW

In [66]:
#salvando e exibindo resultados
vetor,dicionario = Bow(teste_pipeline["data"])
dicionario = list(dicionario.word_index.keys())
print(dicionario)
print(vetor)
print(vetor[:, 0])

['ibm', 'ir', 'empresa', 'tecnologia', 'brasil', 'modelo', 'inteligencia', 'artificial', 'tambem', 'ser', 'generativo', 'ano', 'usp', 'dar', 'ha', 'watsonx', 'nao', 'uso', 'quantico', 'ajudar', 'digital', 'dado', 'ja', 'negocio', 'computacao', 'noticio', 'cliente', 'sao', 'ai', 'politico', 'estao', 'trabalho', 'brasileiro', 'estar', 'hora', 'falar', 'mercado', 'ate', 'aler', 'privacidade', 'computador', 'pesquisa', 'consulting', 'anunciar', 'email', 'setor', 'projeto', 'lider', 'r', 'microsoft', 'criar', 'paulo', 'economia', 'direito', 'us', 'caso', 'incluir', 'globo', 'mundo', 'analise', 'ceo', 'servico', 'indigena', 'processo', 'foto', 'de', 'linguagem', 'abrir', 'conteudo', 'trabalhar', 'voce', 'watson', 'apresentar', 'azure', 'c', 'chip', 'mostrar', 'bradesco', 'plataforma', 'dia', 'google', 'experiencia', 'evento', 'possivel', 'fazer', 'comentario', 'newsletter', 'reservar', 'openai', 'financeiro', 'banco', 'adobe', 'desenvolver', 'ver', 'bom', 'conjunto', 'principal', 'executivo'

## 8.5 Visualização do modelo

In [67]:
#vetor
vetorexportação = vetor[:, 1:]
#para conseguir ver e exportar
dfBOW = pd.DataFrame(vetorexportação, columns=dicionario, index=teste_pipeline)
dfBOW

,ibm,ir,empresa,tecnologia,brasil,modelo,inteligencia,artificial,tambem,ser,...,integracao,globalmente,ivone,santano,17h17,qualificacao,envolverar,veracel,monkey,fornecedor
"([ibm, falar, avanco, Watsonx, plataformar, ai, empresa, bruno, Martinez, agosto, empresa, mostrar, resultado, bia, assistente, pessoal, bradesco, comentar, inteligencia, artificial, ja, ajudar, empresa, parceiro, indice, ibm, ajudar, empresa, entrar, mundo, ai, Watsonx, Watsonxdata, Watsonxgovernance, gestor, enfrentar, desafio, ligar, gestao, dar, empresa, durante, evento, fechar, jornalista, ibm, falar, importanciar, inteligencia, artificial, generativo, tambem, chamar, genai, ajudar, empresa, diversos, mercado, especifico, Watsonx, utilizar, diversos, caso, sucesso, bradesco, desenvolver, assistente, pessoal, bia, detalhe, ibm, ajudar, empresa, entrar, mundo, ai, empresa, ser, solicitar, ajudar, hardware, estao, ser, contratar, tambem, aplicacao, inteligencia, artificial, diversos, processo, caso, voce, nao, saber, ir, generativa, utilizar, diversos, situacoes, realizar, ...],)",20.0,9.0,25.0,5.0,4.0,4.0,12.0,12.0,19.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"([ibm, microsoft, expander, parceria, oferecer, solucoes, ir, generativo, oferta, ibm, consulting, Azure, openai, service, disponivel, azure, marketplace, servico, ir, totalmente, gerenciar, redacao, agosto, pm, compartilhar, ibm, anunciar, quintafeira, expandir, colaboracao, microsoft, ajudar, cliente, conjunto, acelerar, implementacao, ir, generativa, oferecer, oferta, fornecer, cliente, experiencia, tecnologia, precisar, inovar, processo, negocio, dimensionar, ir, maneira, eficaz, ibm, consulting, colaboracao, microsoft, concentrar, ajudar, cliente, implementar, dimensionar, azure, openai, service, oferto, ibm, consulting, Azure, openai, service, disponivel, azure, marketplace, servico, ir, totalmente, gerenciar, permitir, desenvolvedore, cientista, dado, apliquir, modelo, linguagem, poderoso, incluir, serie, gpt, codex, objetivo, ajudar, empresa, definir, estrategia, adocao, conjunto, inicial, caso, uso, ir, ...],)",18.0,22.0,5.0,11.0,0.0,6.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"([pioneirar, ir, ibm, mostrar, estrategio, corrido, linguagem, generativo, apresentar, pioneira, ir, ibm, mostrar, estrategia, corrido, linguagem, generativo, fabio, Lima, cientista, dar, companhia, Brasil, falar, lancamento, Watsonx, tecnologia, mudar, dinamica, negocio, pacete, julho, atualizar, ha, compartilhe, publicacao, fabio, lima, estar, de, aquele, raro, momento, historia, chegar, inovacao, tecnologico, transforma, radicalmente, negocio, sociedade, acessibilidade, l, l, tema, inteligencia, artificial, ibm, costumar, primeiro, empresa, lembrar, comum, discussoe, relacao, pessoa, x, maquinas, surja, batalha, deepr, blue, supercomputador, criar, companhia, Campeao, Mundo, xadrez, garryr, kasparov, kasparov, ganhar, tr, partida, empatar, perder, deep, blue, deepr, blue, tecnologia, antecessora, watson, principal, plataforma, inteligencia, artificial, ibm, atualmente, tro, ...],)",13.0,20.0,7.0,6.0,5.0,15.0,5.0,5.0,2.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"([inteligencia, artificial, ibm, comentar, jogo, torneio, us, Open, tenis, longo, torneio, tecnologia, ir, automatizara, producao, narracoes, audio, detalhar, legenda, acompanhar, video, bom, momento, torneio, redacao, agosto, am, compartilhar, ibm, associacao, tenis, Estados, Unidos, ust, anunciar, funcionalidade, digital, aparecerao, usopenorg, aplicativo, us, Open, fas, incluir, comentario, falar, gerar, inteligencia, artificial, ir, desenvolver, oferecer, fas, experiencia, perspicaz, engajadora, atualizar, momento, importante, torneio, ibm, Watsonx, powered, ai, commentary, modelo, construir, treinar, implementado, Watsonx, plataforma, dar, ir, ibm, longo, torneio, tecnologia, ir, automatizara, producao, narracoes, audio, detalhar, legenda, acompanhar, video, bom, momento, us, Open, escalar, precedente, partida, sor

In [68]:
def contarPalavras(dicionario, matriz):
    dicionario = [''] + dicionario
    contagemPalavras = [0] * len(dicionario)

    for i, palavra in enumerate(dicionario):
        for j in range(len(matriz)):
            contagemPalavras[i] += matriz[j][i]

    contagemDf = pd.DataFrame({'palavra': dicionario, 'repetição': contagemPalavras})

    return contagemDf.iloc[1:]

In [69]:
# Geração da tabela de contagem
contador = contarPalavras(dicionario, vetor)

In [70]:
# Exibição das primeiras 10 palavras mais recorrentes no Bag Of Words
contador.head(10)

,palavra,repetição
1,ibm,210.0
2,ir,173.0
3,empresa,110.0
4,tecnologia,97.0
5,brasil,77.0
6,modelo,65.0
7,inteligencia,64.0
8,artificial,63.0
9,tambem,61.0
10,ser,57.0


# 9.0 TF-IDF

Frequência do termo (TF) – é a frequência das palavras em um texto de amostra.

Frequência inversa do documento (IDF) – destaca a frequência das palavras em outros textos de amostra. Os recursos são raros ou comuns nos textos de exemplo é a principal preocupação aqui.

Quando usamos ambos os TF-IDF juntos (TF * IDF), as palavras de alta frequência em um texto de exemplo que tem baixa ocorrência em outros textos de exemplo recebem maior importância.

In [71]:
df_tfidf = pd.DataFrame()

df_tfidf['data'] = df_url['url_noticia'].apply(remove_tags)

df_tfidf['data'] = stopWords(df_tfidf['data'])

https://www.showmetech.com.br/ibm-mostra-avancos-da-watsonx/
https://inforchannel.com.br/2023/08/17/ibm-e-microsoft-expandem-parceria-para-oferecer-novas-solucoes-de-ia-generativa/
https://forbes.com.br/forbes-tech/2023/07/pioneira-em-ia-ibm-mostra-sua-estrategia-na-corrida-da-linguagem-generativa/
https://inforchannel.com.br/2023/08/19/inteligencia-artificial-da-ibm-comentara-jogos-no-torneio-us-open-de-tenis/
https://itforum.com.br/ibm-quer-ampliar-base-watsonx/
https://www.em.com.br/app/noticia/gerais/2023/09/07/interna_gerais,1558346/candidatos-de-minas-gerais-teriam-sido-excluidos-de-selecao-da-ibm.shtml
https://epocanegocios.globo.com/futuro-do-dinheiro/noticia/2023/08/bradesco-e-itau-se-aliam-a-ibm-para-testar-uso-da-computacao-quantica-no-mercado-financeiro.ghtml
https://tecmasters.com.br/ceo-openai-diretora-ibm-depoimento-senado/#:~:text=CEO%20da%20OpenAI%20e%20diretora%20da%20IBM%20prestam%20depoimento%20ao%20Senado,-Audi%C3%AAncia%20est%C3%A1%20marcada&text=Enquanto%20o%20Co

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [73]:
tfidf_vectorizer = TfidfVectorizer()

In [74]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df_tfidf['data'])

In [75]:
# Obtém o nome das palavras (termos)
terms = tfidf_vectorizer.get_feature_names_out()

# Calcula a média TF-IDF para cada termo em todas as notícias
tfidf_means = tfidf_matrix.mean(axis=0).tolist()[0]

# Cria um DataFrame com os termos e suas pontuações TF-IDF médias
tfidf_df = pd.DataFrame({'Termo': terms, 'TF-IDF Médio': tfidf_means})

# Classifica o DataFrame por pontuação TF-IDF média em ordem decrescente
tfidf_df = tfidf_df.sort_values(by='TF-IDF Médio', ascending=False)

# Exibe as principais palavras-chave (por exemplo, as 20 principais)
print(tfidf_df.head(20))

             Termo  TF-IDF Médio
1809           ibm      0.116717
1807            ia      0.108564
3737       watsonx      0.054925
1246      empresas      0.052079
978          dados      0.052040
1680    generativa      0.049412
1752            ha      0.048279
482         brasil      0.047969
291     artificial      0.046480
3431    tecnologia      0.046016
1951  inteligencia      0.045657
2342       modelos      0.040289
2306     microsoft      0.039539
1789         horas      0.038667
3413        tambem      0.033266
2415           nao      0.033058
3611           usp      0.033046
402          azure      0.032960
105             ai      0.031703
2294          meta      0.030031


# 11.0 Word2vec com modelo pré treinado

O Word2Vec é uma técnica de Processamento de Linguagem Natural (PLN) que transforma palavras em vetores numéricos. A principal ideia por trás do Word2Vec é que palavras semelhantes em contexto terão representações vetoriais semelhantes, ou seja, vetores próximos em um espaço multidimensional. Isso permite que as palavras sejam representadas de forma densa e contínua, capturando informações semânticas e relacionamentos entre elas.



## 11.1 CBOW (Continuous Bag of Words)

O W2V é uma técnica que mapeia palavras para vetores densos de números reais. O CBOW é um modelo W2V que prevê a palavra alvo com base em seu contexto, ou seja, palavras próximas a ela em uma frase. Essa técnica captura relações semânticas entre palavras.

In [76]:
teste_pipeline_word2Vec = pipeline_dataframe(df_pipeline, "url_noticia")

https://www.showmetech.com.br/ibm-mostra-avancos-da-watsonx/
https://inforchannel.com.br/2023/08/17/ibm-e-microsoft-expandem-parceria-para-oferecer-novas-solucoes-de-ia-generativa/
https://forbes.com.br/forbes-tech/2023/07/pioneira-em-ia-ibm-mostra-sua-estrategia-na-corrida-da-linguagem-generativa/
https://inforchannel.com.br/2023/08/19/inteligencia-artificial-da-ibm-comentara-jogos-no-torneio-us-open-de-tenis/
https://itforum.com.br/ibm-quer-ampliar-base-watsonx/
https://www.em.com.br/app/noticia/gerais/2023/09/07/interna_gerais,1558346/candidatos-de-minas-gerais-teriam-sido-excluidos-de-selecao-da-ibm.shtml
https://epocanegocios.globo.com/futuro-do-dinheiro/noticia/2023/08/bradesco-e-itau-se-aliam-a-ibm-para-testar-uso-da-computacao-quantica-no-mercado-financeiro.ghtml
https://tecmasters.com.br/ceo-openai-diretora-ibm-depoimento-senado/#:~:text=CEO%20da%20OpenAI%20e%20diretora%20da%20IBM%20prestam%20depoimento%20ao%20Senado,-Audi%C3%AAncia%20est%C3%A1%20marcada&text=Enquanto%20o%20Co

In [77]:
teste_pipeline_word2Vec

,data
0,"[ibm, falar, avanco, Watsonx, plataformar, ai,..."
1,"[ibm, microsoft, expander, parceria, oferecer,..."
2,"[pioneirar, ir, ibm, mostrar, estrategio, corr..."
3,"[inteligencia, artificial, ibm, comentar, jogo..."
4,"[ibm, ampliar, base, cliente, watson, Watsonx,..."
5,"[candidato, mina, geral, ter, ser, excluir, se..."
6,"[bradesco, itaur, al, ibm, testar, uso, comput..."
7,"[ceo, openai, diretora, ibm, prestam, depoimen..."
8,"[ibm, lancar, llama, meta, plataformo, ir, emp..."
9,"[ibm, lanca, chip, ir, analogico, inspirar, ce..."


Carregando um modelo pré-treinado de Word2Vec

In [78]:
# Word2vec 50 Dimensões
modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/Shareddrives/CogniVoice/cbow_s50.txt")

A função abaico utiliza o modelo pré-treinado Word2Vec para calcular vetores de palavras para cada frase na coluna de entrada, calculando a média dos vetores das palavras em cada frase. Isso pode ser usado para representar as frases em um espaço vetorial contínuo, onde a similaridade entre vetores reflete a semelhança semântica entre as frases.

In [79]:
def word2Vec(coluna):
  x = []
  for i in range(0,len(coluna)):
    vetor = []
    for h in range(0,len(coluna[i])):
      # Adiciona o vetor da palavra na posição h (índice h) da frase atual no vetor temporário
      vetor.append(modelo_cbow[h])
    # Após percorrer todas as palavras da frase e construir o vetor temporário, este trecho de código calcula a média dos vetores das palavras na frase e adiciona esse vetor médio à lista x.
    x.append(list(map(sum, zip(*vetor))))
    vetor=[]
  print(x)
  return x

In [80]:
testeword2Vec = word2Vec(teste_pipeline_word2Vec['data'])
testeword2Vec[0]
dfTesteword2Vec = pd.DataFrame(testeword2Vec)
dfTesteword2Vec

[[29.229859886676422, 23.144124344165903, -41.131239428061235, -6.932935793156503, 46.01373738082475, 45.06564345333027, 21.577802930783946, 28.23191865766421, 55.11025614178652, 57.57204786170041, -49.248572896751284, 11.000659827317577, -30.612349015042128, 36.68970000476111, -3.3604570261086337, 77.70832308805257, -25.57046619360335, -58.976716191566084, -179.95247940719128, 51.396857449319214, -125.16462724801386, 106.83804113566293, -80.97991178226948, 114.49491802857665, 11.43884714266187, -38.78722905291943, 2.5510060011729365, -36.78648780868389, -15.08718627266353, 55.09274822371299, 106.21737521077739, -74.6714558590902, 59.25822789652739, -160.56699611696604, 4.880705945019145, 32.01747924700612, 36.97418284736341, -79.71066735238128, 5.948824087987305, 42.46010386607122, 34.02169940742897, 125.77946435521153, -43.89794860069014, 48.840891771178576, 52.69815744733205, 0.12097035473561846, 14.259961055640815, -34.99055807563127, -98.6058812331903, -33.146563289657934], [13.73

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,29.229860,23.144124,-41.131239,-6.932936,46.013737,45.065643,21.577803,28.231919,55.110256,57.572048,...,34.021699,125.779464,-43.897949,48.840892,52.698157,0.120970,14.259961,-34.990558,-98.605881,-33.146563
1,13.739605,8.178894,-34.609981,-9.121365,39.255959,36.151091,6.830879,12.197875,39.358064,40.782039,...,25.806639,88.087382,-42.068928,30.137091,39.795897,6.614877,11.386024,-30.747539,-73.458774,-27.579839
2,5.104322,-1.234588,-21.847951,-16.300693,28.248550,25.583675,6.168330,11.073787,29.564538,24.562591,...,15.818654,56.811564,-35.550641,19.630719,31.782201,4.643000,12.729313,-20.744882,-51.620646,-20.525793
3,12.222293,6.042495,-32.416635,-10.751904,37.386172,32.453893,7.213328,13.651352,35.851206,35.341828,...,23.397405,83.864302,-40.282230,28.490593,34.864128,3.034807,14.662928,-28.631766,-68.966275,-26.148100
4,5.423434,0.082728,-22.528805,-15.495416,28.964577,25.529735,6.470585,10.455556,30.298490,26.372371,...,15.420025,59.611113,-35.104929,20.645352,34.083919,3.114444,12.939250,-23.780918,-55.325182,-21.684936
5,-3.073257,-9.882277,-19.343451,-15.890033,22.077090,18.837668,3.236981,9.236344,19.762412,8.672978,...,6.640214,31.683968,-30.349540,13.345251,29.205200,9.665801,6.056548,-10.671688,-39.074615,-10.185493
6,43.534427,55.592801,-51.845438,-7.019375,63.696132,54.528472,45.246527,43.999177,77.761340,87.218384,...,48.126953,180.832398,-47.852574,73.767250,75.167080,-8.287920,24.777016,-41.179815,-137.392474,-52.672196
7,1.230970,-7.817092,-23.026355,-18.427341,23.516629,21.685860,5.317932,11.915139,26.771056,16.539601,...,9.746995,40.346940,-33.732230,17.155841,28.706143,7.795733,9.282026,-15.701799,-40.233007,-12.643719
8,-1.609146,-10.786071,-7.641823,-11.990607,10.839929,-1.319044,-3.118088,4.143900,-0.303141,-7.268402,...,-2.456309,-0.015868,-11.678691,1.388872,15.584552,11.687938,-0.985810,-3.243614,-19.365446,0.845256
9,-0.013373,-5.795232,-25.711536,-18.852083,22.968357,22.930896,7.335692,12.103488,25.906650,18.685462,...,9.838373,42.796238,-34.208993,18.098979,28.304547,6.935699,10.573702,-14.393928,-41.399362,-13.234082


In [81]:
phrase_input = "Projeto que utiliza inteligência artificial pretende fortalecer línguas indígenas no Brasil envolvendo nossas terras e os europeus"
phrase_input = pipeline_speech_to_text(phrase_input)
phrase_input

[['projeto',
  'utilizar',
  'inteligencia',
  'artificial',
  'pretender',
  'fortalecer',
  'lingua',
  'indigena',
  'Brasil',
  'envolver',
  'terra',
  'europeu']]

In [82]:
testeword2Vecc_ai_input = word2Vec(phrase_input)
dfTesteword2Vecc_ai_input = pd.DataFrame(testeword2Vecc_ai_input)
dfTesteword2Vecc_ai_input

[[0.9874520055018365, -1.0998779768124223, 0.10618801091914065, -1.99051803490147, 1.079730013385415, -0.2301399951102212, -0.1674950048327446, -1.580196981318295, -2.1343939874786884, -1.5771940046688542, 1.8618619812186807, -0.3940339726395905, 0.8302629599347711, -0.4832860061433166, 0.6199799960013479, -0.8383390069939196, 0.6138660162687302, 0.8312119897454977, 2.496050999732688, -0.6091510187834501, 1.2750869831070304, -1.4337360057979822, 0.30009501006861683, -1.9298830106854439, -1.019969005137682, -0.3003050144761801, -0.10966201033443213, -0.38055399572476745, 0.33479201467707753, -1.2477209768258035, -1.1612840352463536, -0.09554300853051245, -1.1571439940016717, 1.3279450095724314, 0.8990799856837839, -0.004901024512946606, 0.30776399932801723, 0.8577430173754692, -0.2721709879115224, -1.6351239969953895, 0.3293770137242973, -2.356407980900258, 0.41592699754983187, -0.35470499680377543, 0.5829000002704561, 0.33708896208554506, -0.9396099904552102, 0.6648110051173717, 0.9433

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.987452,-1.099878,0.106188,-1.990518,1.07973,-0.23014,-0.167495,-1.580197,-2.134394,-1.577194,...,0.329377,-2.356408,0.415927,-0.354705,0.5829,0.337089,-0.93961,0.664811,0.943313,1.366819


## 11.2 Word2Vec com Skipgram

Similar ao CBOW, o modelo de Skipgram é uma abordagem no W2V que faz o oposto. Ele prevê as palavras vizinhas com base em uma palavra de destino. Também é eficaz na captura de relações semânticas entre palavras.

In [83]:
# SkipGram 50 Dimensões
modelo_skipgram = KeyedVectors.load_word2vec_format("/content/drive/Shareddrives/CogniVoice/skip_s50.txt")

In [84]:
def word2Vec_skipgram(coluna):
    # Lista que armazena os vetores resultantes de todas as frases
    x = []
    for i in range(len(coluna)):
        # Lista que armazena os vetores das palavras de cada frase
        vetor = []
        for palavra in coluna[i]:
            if palavra in modelo_skipgram:
                vetor.append(modelo_skipgram[palavra])
        if vetor:
             # Soma dos vetores de todas as palavras
            x.append(list(map(sum, zip(*vetor))))
    return x

In [85]:
testeword2Vec_skipgram = word2Vec_skipgram(teste_pipeline_word2Vec['data'])
testeword2Vec_skipgram[0]
dfTesteword2Vec_skipgram = pd.DataFrame(testeword2Vec_skipgram)
dfTesteword2Vec_skipgram

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-171.498996,-106.437446,52.128961,-31.566868,-108.122325,65.329388,-143.056576,75.917109,-66.255517,42.868410,...,83.289517,-66.182504,-13.557832,150.370861,-41.968517,-59.382496,99.062248,156.728011,142.455529,128.292858
1,-117.474145,-95.505278,70.799589,-34.358676,-95.860670,72.321047,-142.946339,39.087510,-89.784451,56.366166,...,82.925318,-51.096455,-2.782374,105.107241,10.867781,-47.240750,91.320595,159.592489,158.582038,111.908911
2,-91.516552,-52.656226,27.533587,-16.090328,-54.984831,27.804459,-85.085944,32.057246,-33.606473,13.999285,...,45.803342,-32.654953,-15.050032,82.725032,-23.747081,-23.531555,44.735716,91.024721,68.039156,62.151866
3,-71.297039,-77.147229,63.735889,-40.785071,-108.817968,72.623668,-105.649104,30.096128,-107.217128,1.100962,...,67.320295,-18.401546,-22.201318,85.254780,18.785574,-56.755077,97.974135,109.987611,146.924339,88.199158
4,-92.768363,-56.752069,33.280915,-25.450230,-71.732466,32.366373,-77.476901,30.648579,-48.608047,9.970663,...,42.690943,-29.706976,9.371969,82.444024,-7.374708,-38.266863,50.136870,97.206412,78.616673,75.563978
5,-52.065609,-26.103945,-12.023296,-18.623879,-54.647441,6.405816,-27.246301,35.669562,-35.398164,2.464673,...,35.993227,-18.685496,-5.411590,61.700913,-18.831454,-29.876871,65.998287,73.861009,65.837427,38.615821
6,-180.531273,-104.568249,87.265061,-56.708246,-180.495821,125.519772,-197.456953,28.197607,-110.719121,18.274637,...,117.475300,-56.356538,24.854263,177.265833,-56.268361,-56.300035,173.520001,215.230265,166.838368,157.853665
7,-41.966646,-45.096337,28.433743,-42.243844,-85.494532,20.578227,-35.422493,26.858452,-69.248987,-6.262041,...,22.516552,47.387110,-19.210429,62.933595,39.852646,-20.063221,69.107979,63.612475,119.039121,70.678006
8,-28.725592,-20.994639,12.964035,-5.918352,-25.317758,16.035425,-37.002177,9.556427,-28.512536,6.650666,...,18.386693,-21.108357,-14.476060,30.993046,-1.640568,0.609824,30.069405,40.030882,37.147241,27.989091
9,-81.301436,-57.537742,37.237299,-22.245534,-53.664566,27.220539,-80.985820,16.391046,-42.928244,20.782713,...,27.257518,-29.899684,-20.688406,84.828160,-7.967166,-5.421147,57.592650,94.121376,73.524405,60.605660


# 12.0 Definição da função de embedding

Uma função de embedding é uma técnica utilizada no campo de Processamento de Linguagem Natural (PLN) para mapear elementos discretos, como palavras ou itens, em vetores de números reais (também conhecidos como embeddings). Esses vetores representam as características e relações semânticas entre os elementos no espaço vetorial.

O objetivo principal de uma função de embedding é criar uma representação densa e contínua para os elementos de um vocabulário, de modo que elementos semanticamente semelhantes sejam mapeados para vetores próximos no espaço vetorial. Isso permite que modelos de aprendizado de máquina entendam e processem esses elementos de forma mais eficaz, capturando informações semânticas e relacionamentos entre eles.

Nesse projeto utilizaremos as seguintes função de ebedding com as seguintes técnicas:

1.   Transformers
2.   BERT

In [86]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6003a09ac39c0b9ecf9d2a6be10f34b556d596076844b8216343521310b5d45c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [87]:
from sentence_transformers import SentenceTransformer
from transformers import BertForSequenceClassification

In [88]:
#função para vetorização
def Vetorizacao(dataframe,modelo):
  # Calcula os embeddings para a coluna de texto
  embeddings_liz = modelo.encode(dataframe.tolist())

  # Cria um novo DataFrame com os embeddings
  embeddings_df = pd.DataFrame(embeddings_liz)

  return embeddings_df

# 12.0 Transformers vetorização

Os modelos Transformers, como o GPT (Generative Pre-trained Transformer) e o BERT (Bidirectional Encoder Representations from Transformers), são arquiteturas de aprendizado de máquina que revolucionaram o PLN. Eles são capazes de considerar o contexto bidirecional de palavras em uma sentença, capturando informações contextuais de maneira mais eficaz.

In [89]:
vetorizadorTransformers = SentenceTransformer('distiluse-base-multilingual-cased')

In [90]:
vetoresTransformers = Vetorizacao(teste_pipeline_word2Vec['data'], vetorizadorTransformers)

In [91]:
vetoresTransformers

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.028666,0.028220,-0.023384,0.014083,0.007305,0.017510,-0.044739,-0.050912,-0.038454,-0.017217,...,0.038639,-0.008759,0.006478,-0.009722,-0.005133,-0.042704,0.012560,-0.022124,0.004717,-0.055538
1,0.010163,-0.001450,0.039733,0.006941,-0.000967,0.025527,-0.054160,-0.008372,-0.017906,-0.002977,...,0.056116,0.054105,0.026207,0.001357,-0.003072,-0.035279,-0.023086,-0.034095,0.005613,-0.031090
2,0.015046,0.001065,-0.032011,-0.024556,-0.016283,0.011762,-0.017354,-0.006812,-0.025012,-0.013548,...,0.017461,-0.023936,-0.003925,0.007662,-0.029126,-0.024618,-0.004561,-0.011405,-0.004372,-0.003237
3,-0.029894,0.017164,0.001487,0.007057,-0.007762,0.027223,-0.021587,-0.006010,-0.067396,-0.004103,...,0.004324,-0.004974,-0.006747,0.028096,-0.065249,-0.112144,-0.041665,0.022882,0.005654,-0.029399
4,0.008938,0.015493,-0.006235,0.035471,0.006020,0.045250,-0.017286,-0.040471,-0.039655,-0.007309,...,0.031619,0.013155,-0.005147,0.008457,-0.011148,-0.033835,-0.003118,-0.076486,0.024971,0.016575
5,0.021153,0.006390,-0.029137,-0.007356,-0.014631,0.014365,-0.011096,-0.003775,-0.022197,-0.019609,...,0.036635,0.000865,0.003886,0.008693,-0.046268,-0.049831,-0.013253,-0.006586,-0.007210,-0.014088
6,0.017568,-0.024916,0.015976,-0.014891,-0.044066,-0.007900,0.012807,-0.016093,-0.005097,-0.015958,...,0.046724,-0.032696,0.008206,0.000092,-0.041121,-0.024186,-0.044787,-0.019188,0.034205,0.015779
7,0.023909,-0.001399,-0.044014,-0.021167,-0.006869,0.009037,-0.045330,-0.060539,-0.003867,-0.004215,...,0.028462,-0.018991,0.021015,-0.001583,-0.019536,-0.017290,-0.004180,-0.035361,-0.046656,0.009936
8,0.034297,0.011807,-0.008040,0.004366,0.000280,0.001837,-0.034384,-0.049088,-0.028607,-0.025749,...,0.053416,0.009864,0.021292,-0.006798,-0.001078,-0.034914,-0.000685,-0.045522,-0.000403,-0.023170
9,0.012067,0.022410,-0.017147,0.002757,-0.008921,-0.004622,-0.021052,-0.034417,-0.023519,-0.011880,...,0.046737,-0.004118,0.003143,-0.000834,-0.007140,-0.030478,-0.015171,-0.043992,0.004189,-0.027616


In [92]:
phrase_input_transformers = "Projeto que utiliza inteligência artificial pretende fortalecer línguas indígenas no Brasil envolvendo nossas terras e os europeus"
phrase_input_transformers = pipeline_speech_to_text(phrase_input_transformers)
phrase_input_transformers

[['projeto',
  'utilizar',
  'inteligencia',
  'artificial',
  'pretender',
  'fortalecer',
  'lingua',
  'indigena',
  'Brasil',
  'envolver',
  'terra',
  'europeu']]

In [93]:
breno = vetorizadorTransformers.encode(phrase_input_transformers)

# 13.0 Vetorizadro Bert

O BERT é um modelo de linguagem pré-treinado baseado em Transformers que alcançou resultados de estado-da-arte em uma variedade de tarefas de PLN. Ele é treinado para prever palavras em uma sentença, considerando todas as palavras em ambos os lados do contexto, tornando-o altamente eficaz na compreensão do significado contextual das palavras.

In [94]:
vetorizadorBert = SentenceTransformer('bert-base-multilingual-cased')

In [95]:
vetoresBert = Vetorizacao(teste_pipeline_word2Vec['data'], vetorizadorBert)

In [96]:
vetoresBert

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.037571,-0.134126,0.583761,0.112654,0.204770,0.178488,0.142073,-0.173217,0.089179,0.331715,...,-0.109005,0.191583,-0.304709,-0.552582,0.179850,-0.083643,0.274027,-0.064540,0.006353,-0.208390
1,0.047287,-0.406079,0.696057,0.107339,0.348362,0.124650,-0.212150,-0.176422,0.112529,0.155711,...,-0.054580,0.140933,-0.343495,-0.732228,0.260813,-0.148524,0.218278,0.023073,-0.179050,-0.039204
2,0.006295,-0.038772,0.472723,0.002407,0.060892,0.180560,0.001610,-0.019124,-0.017071,0.334687,...,0.009681,0.337262,-0.259232,-0.398253,0.049896,0.188736,0.151854,-0.137689,0.103421,0.031288
3,-0.052546,-0.290894,0.408679,0.023871,-0.245646,0.011187,-0.039897,0.014946,0.070618,0.346659,...,0.124947,0.177349,-0.449140,-0.142837,0.035818,0.094336,0.133570,0.056862,0.003069,-0.225844
4,0.091002,-0.275092,0.614563,0.142003,0.248706,0.218308,0.196526,-0.148189,0.021342,0.237373,...,-0.168077,0.146152,-0.229417,-0.617866,0.204632,-0.057292,0.241719,0.012886,0.073412,-0.137627
5,0.166029,-0.240711,0.633223,0.132679,0.073813,-0.055811,0.184122,-0.011314,0.001667,0.309815,...,-0.097076,0.310721,-0.240146,-0.292015,0.214166,0.072483,0.145336,0.041870,-0.065844,-0.008681
6,0.019097,-0.130792,0.686319,0.080190,0.135623,-0.164267,-0.059922,-0.006718,0.060772,0.351402,...,-0.065376,0.511272,-0.223821,-0.381608,0.141807,-0.115576,0.055948,-0.056580,-0.022582,0.008352
7,0.232904,-0.139818,0.706019,0.031965,0.108839,-0.047334,-0.105564,-0.174127,-0.157285,0.270049,...,-0.097537,0.411786,-0.396843,-0.440783,0.175776,-0.176976,0.194601,-0.120256,0.055828,0.040554
8,0.183132,-0.193198,0.529429,0.123516,0.272666,0.113360,0.029198,-0.103609,-0.079157,0.274650,...,-0.248267,0.360554,-0.170745,-0.566018,0.254850,-0.228539,0.162526,0.078295,-0.052866,0.060981
9,0.123641,-0.236952,0.565839,0.150594,0.332262,0.056304,-0.028292,-0.132892,-0.021003,0.351959,...,-0.263379,0.351623,-0.135058,-0.561452,0.219605,-0.247362,0.230648,0.017534,-0.023004,0.038001


In [97]:
phrase_input_bert = "Projeto que utiliza inteligência artificial pretende fortalecer línguas indígenas no Brasil envolvendo nossas terras e os europeus"
phrase_input_bert = pipeline_speech_to_text(phrase_input_bert)
phrase_input_bert

[['projeto',
  'utilizar',
  'inteligencia',
  'artificial',
  'pretender',
  'fortalecer',
  'lingua',
  'indigena',
  'Brasil',
  'envolver',
  'terra',
  'europeu']]

In [98]:
breno_bert = vetorizadorBert.encode(phrase_input_bert)

# 14.0 Similaridade do coseno

A função de similaridade do cosseno é uma métrica usada para medir a semelhança entre dois vetores em um espaço multidimensional. Ela calcula o coseno do ângulo entre esses vetores. Quanto mais próximos os vetores estiverem um do outro, menor será o ângulo e, portanto, maior será o valor do cosseno, indicando maior similaridade. Por outro lado, se os vetores estiverem em direções opostas, o cosseno será -1, indicando dissimilaridade completa. O valor da similaridade do cosseno varia de -1 (completamente dissimilar) a 1 (completamente similar), com 0 indicando independência.

Vamos empregar a função de similaridade do cosseno para comparar o input com todas as notícias armazenadas no banco de dados, que também foram vetorizadas. Com os resultados obtidos através dessa função, identificaremos quais notícias são mais semelhantes ao input, permitindo-nos apresentar as notícias relevantes ao usuário.

In [99]:
def calculate_similiarity(vectors: list, phrase_vector: list) -> float:
    text_similarity_array = []

    if str(type(phrase_vector)) != "<class 'numpy.ndarray'>":
      phrase_vector = phrase_vector.values
    else:
      pass

    for index, row in vectors.iterrows():
        np_vector_row = row.values
        np_vector_text = phrase_vector

        product_sum = np.sum(np_vector_row * np_vector_text)

        squared_frase_um = np.square(np_vector_row)
        squared_frase_dois = np.square(np_vector_text)

        sum_of_squares_array1 = np.sum(squared_frase_um)
        sum_of_squares_array2 = np.sum(squared_frase_dois)

        modulo = math.sqrt(sum_of_squares_array1) * math.sqrt(sum_of_squares_array2)

        text_similarity = product_sum / modulo

        text_similarity_array.append(text_similarity)


    return text_similarity_array

#15.0 Casos de testes - Similaridade da noticia com input

Nesta seção, realizamos testes de vetorização e cálculo de similaridade utilizando todos os métodos previamente abordados, com o objetivo de determinar qual modelo pré-treinado melhor se adapta à nossa base de dados. É importante destacar que optamos por não incluir o modelo Bag of Words nestes testes, devido às particularidades da nossa base de dados.

Nossa base de dados consiste em notícias frequentemente extensas, o que resulta em um dimensionamento massivo do Bag of Words, tornando-o computacionalmente ineficiente. Além disso, devido à natureza variável das entradas dos usuários, surgiria a necessidade de criar um novo dicionário a cada novo input do usuário, o que seria impraticável em termos de tempo e recursos computacionais.

Portanto, ao realizar esses testes de vetorização e cálculo de similaridade, estamos buscando determinar qual modelo pré-treinado oferece o melhor desempenho em nossa base de dados, considerando tanto a eficácia quanto a eficiência computacional. Essa análise nos ajudará a escolher a abordagem mais adequada para a nossa aplicação específica.

## Definição da tabela e input

In [100]:
df_test_vectors_copy = pd.read_csv('/content/drive/Shareddrives/CogniVoice/ibm_db.csv')
df_test_vectors = pipeline_dataframe(df_test_vectors_copy, "url_noticia")
df_test_vectors

https://www.showmetech.com.br/ibm-mostra-avancos-da-watsonx/
https://inforchannel.com.br/2023/08/17/ibm-e-microsoft-expandem-parceria-para-oferecer-novas-solucoes-de-ia-generativa/
https://forbes.com.br/forbes-tech/2023/07/pioneira-em-ia-ibm-mostra-sua-estrategia-na-corrida-da-linguagem-generativa/
https://inforchannel.com.br/2023/08/19/inteligencia-artificial-da-ibm-comentara-jogos-no-torneio-us-open-de-tenis/
https://itforum.com.br/ibm-quer-ampliar-base-watsonx/
https://www.em.com.br/app/noticia/gerais/2023/09/07/interna_gerais,1558346/candidatos-de-minas-gerais-teriam-sido-excluidos-de-selecao-da-ibm.shtml
https://epocanegocios.globo.com/futuro-do-dinheiro/noticia/2023/08/bradesco-e-itau-se-aliam-a-ibm-para-testar-uso-da-computacao-quantica-no-mercado-financeiro.ghtml
https://tecmasters.com.br/ceo-openai-diretora-ibm-depoimento-senado/#:~:text=CEO%20da%20OpenAI%20e%20diretora%20da%20IBM%20prestam%20depoimento%20ao%20Senado,-Audi%C3%AAncia%20est%C3%A1%20marcada&text=Enquanto%20o%20Co

,data
0,"[ibm, falar, avanco, Watsonx, plataformar, ai,..."
1,"[ibm, microsoft, expander, parceria, oferecer,..."
2,"[pioneirar, ir, ibm, mostrar, estrategio, corr..."
3,"[inteligencia, artificial, ibm, comentar, jogo..."
4,"[ibm, ampliar, base, cliente, watson, Watsonx,..."
5,"[candidato, mina, geral, ter, ser, excluir, se..."
6,"[bradesco, itaur, al, ibm, testar, uso, comput..."
7,"[ceo, openai, diretora, ibm, prestam, depoimen..."
8,"[ibm, lancar, llama, meta, plataformo, ir, emp..."
9,"[ibm, lanca, chip, ir, analogico, inspirar, ce..."


In [101]:
phrase_title_news = "qual a parceria da ibm com a microsoft"
phrase_title_news = pipeline_speech_to_text(phrase_title_news)
phrase_title_news

[['parceria', 'ibm', 'microsoft']]

##Casos de testes 1 - Modelo Word2Vec 50 dimensões

In [102]:
vector_full_news_w2v = word2Vec(df_test_vectors['data'])
df_full_news_w2v = pd.DataFrame(vector_full_news_w2v)
df_full_news_w2v

[[29.229859886676422, 23.144124344165903, -41.131239428061235, -6.932935793156503, 46.01373738082475, 45.06564345333027, 21.577802930783946, 28.23191865766421, 55.11025614178652, 57.57204786170041, -49.248572896751284, 11.000659827317577, -30.612349015042128, 36.68970000476111, -3.3604570261086337, 77.70832308805257, -25.57046619360335, -58.976716191566084, -179.95247940719128, 51.396857449319214, -125.16462724801386, 106.83804113566293, -80.97991178226948, 114.49491802857665, 11.43884714266187, -38.78722905291943, 2.5510060011729365, -36.78648780868389, -15.08718627266353, 55.09274822371299, 106.21737521077739, -74.6714558590902, 59.25822789652739, -160.56699611696604, 4.880705945019145, 32.01747924700612, 36.97418284736341, -79.71066735238128, 5.948824087987305, 42.46010386607122, 34.02169940742897, 125.77946435521153, -43.89794860069014, 48.840891771178576, 52.69815744733205, 0.12097035473561846, 14.259961055640815, -34.99055807563127, -98.6058812331903, -33.146563289657934], [13.73

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,29.229860,23.144124,-41.131239,-6.932936,46.013737,45.065643,21.577803,28.231919,55.110256,57.572048,...,34.021699,125.779464,-43.897949,48.840892,52.698157,0.120970,14.259961,-34.990558,-98.605881,-33.146563
1,13.739605,8.178894,-34.609981,-9.121365,39.255959,36.151091,6.830879,12.197875,39.358064,40.782039,...,25.806639,88.087382,-42.068928,30.137091,39.795897,6.614877,11.386024,-30.747539,-73.458774,-27.579839
2,5.104322,-1.234588,-21.847951,-16.300693,28.248550,25.583675,6.168330,11.073787,29.564538,24.562591,...,15.818654,56.811564,-35.550641,19.630719,31.782201,4.643000,12.729313,-20.744882,-51.620646,-20.525793
3,12.222293,6.042495,-32.416635,-10.751904,37.386172,32.453893,7.213328,13.651352,35.851206,35.341828,...,23.397405,83.864302,-40.282230,28.490593,34.864128,3.034807,14.662928,-28.631766,-68.966275,-26.148100
4,5.423434,0.082728,-22.528805,-15.495416,28.964577,25.529735,6.470585,10.455556,30.298490,26.372371,...,15.420025,59.611113,-35.104929,20.645352,34.083919,3.114444,12.939250,-23.780918,-55.325182,-21.684936
5,-3.073257,-9.882277,-19.343451,-15.890033,22.077090,18.837668,3.236981,9.236344,19.762412,8.672978,...,6.640214,31.683968,-30.349540,13.345251,29.205200,9.665801,6.056548,-10.671688,-39.074615,-10.185493
6,43.534427,55.592801,-51.845438,-7.019375,63.696132,54.528472,45.246527,43.999177,77.761340,87.218384,...,48.126953,180.832398,-47.852574,73.767250,75.167080,-8.287920,24.777016,-41.179815,-137.392474,-52.672196
7,1.230970,-7.817092,-23.026355,-18.427341,23.516629,21.685860,5.317932,11.915139,26.771056,16.539601,...,9.746995,40.346940,-33.732230,17.155841,28.706143,7.795733,9.282026,-15.701799,-40.233007,-12.643719
8,-1.609146,-10.786071,-7.641823,-11.990607,10.839929,-1.319044,-3.118088,4.143900,-0.303141,-7.268402,...,-2.456309,-0.015868,-11.678691,1.388872,15.584552,11.687938,-0.985810,-3.243614,-19.365446,0.845256
9,-0.013373,-5.795232,-25.711536,-18.852083,22.968357,22.930896,7.335692,12.103488,25.906650,18.685462,...,9.838373,42.796238,-34.208993,18.098979,28.304547,6.935699,10.573702,-14.393928,-41.399362,-13.234082


In [103]:
vector_phrase_title_news_w2v = word2Vec(phrase_title_news)
df_phrase_title_news_w2v = pd.DataFrame(vector_phrase_title_news_w2v)
df_phrase_title_news_w2v

[[0.04743499867618084, -0.007624988444149494, 0.17372301084105857, -0.4450780148617923, -0.05897699762135744, -0.18259899888653308, -0.07950600329786539, -0.22079600114375353, -0.26524299127049744, -0.34696901158895344, 0.18831599480472505, 0.3215109999291599, 0.15891699492931366, -0.10340499668382108, -0.31171900616027415, -0.5871989955194294, 0.23667099699378014, 0.1608599959872663, 0.4703799926210195, -0.1647079959511757, 0.2987969955429435, -0.0428329911082983, 0.012402001055306755, -0.4534440040588379, -0.17294300301000476, 0.08296300191432238, -0.008891000412404537, -0.09823000011965632, 0.14309299644082785, -0.32422799756750464, -0.11807799589587376, -0.02720199921168387, 0.05669700261205435, 0.43027599225752056, 0.33976198895834386, 0.39708699379116297, 0.2405459936708212, 0.40383200254291296, -0.06164200138300657, -0.37846099864691496, -0.012754999566823244, -0.23809099523350596, 0.3093020087108016, -0.140817997045815, -0.07840599725022912, -0.1801350051537156, 0.0139439988415

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.047435,-0.007625,0.173723,-0.445078,-0.058977,-0.182599,-0.079506,-0.220796,-0.265243,-0.346969,...,-0.012755,-0.238091,0.309302,-0.140818,-0.078406,-0.180135,0.013944,-0.043409,0.288535,0.232324


In [104]:
calculate_similiarity(df_full_news_w2v,df_phrase_title_news_w2v)

[-0.6546830969945799,
 -0.6560141649034624,
 -0.6486752380175478,
 -0.6513418005616549,
 -0.6490522595114344,
 -0.644947108109145,
 -0.6418581457508491,
 -0.6478167988976925,
 -0.330125158811855,
 -0.6488389979257516,
 -0.6233351161567867,
 -0.5330747632271892,
 -0.638192183036096,
 -0.6552206027241091,
 -0.5635350517865393,
 -0.2940520399219361,
 -0.6511793754869024,
 -0.1848905141782304,
 -0.6462017578548866,
 -0.6266912641378144]

##Casos de testes 2 - Modelo SkipGram 50 dimensões

In [105]:
vector_full_news_skipgram = word2Vec_skipgram(df_test_vectors['data'])
df_full_news_skipgram = pd.DataFrame(vector_full_news_skipgram)
df_full_news_skipgram

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-171.498996,-106.437446,52.128961,-31.566868,-108.122325,65.329388,-143.056576,75.917109,-66.255517,42.868410,...,83.289517,-66.182504,-13.557832,150.370861,-41.968517,-59.382496,99.062248,156.728011,142.455529,128.292858
1,-117.474145,-95.505278,70.799589,-34.358676,-95.860670,72.321047,-142.946339,39.087510,-89.784451,56.366166,...,82.925318,-51.096455,-2.782374,105.107241,10.867781,-47.240750,91.320595,159.592489,158.582038,111.908911
2,-91.516552,-52.656226,27.533587,-16.090328,-54.984831,27.804459,-85.085944,32.057246,-33.606473,13.999285,...,45.803342,-32.654953,-15.050032,82.725032,-23.747081,-23.531555,44.735716,91.024721,68.039156,62.151866
3,-71.297039,-77.147229,63.735889,-40.785071,-108.817968,72.623668,-105.649104,30.096128,-107.217128,1.100962,...,67.320295,-18.401546,-22.201318,85.254780,18.785574,-56.755077,97.974135,109.987611,146.924339,88.199158
4,-92.768363,-56.752069,33.280915,-25.450230,-71.732466,32.366373,-77.476901,30.648579,-48.608047,9.970663,...,42.690943,-29.706976,9.371969,82.444024,-7.374708,-38.266863,50.136870,97.206412,78.616673,75.563978
5,-52.065609,-26.103945,-12.023296,-18.623879,-54.647441,6.405816,-27.246301,35.669562,-35.398164,2.464673,...,35.993227,-18.685496,-5.411590,61.700913,-18.831454,-29.876871,65.998287,73.861009,65.837427,38.615821
6,-180.531273,-104.568249,87.265061,-56.708246,-180.495821,125.519772,-197.456953,28.197607,-110.719121,18.274637,...,117.475300,-56.356538,24.854263,177.265833,-56.268361,-56.300035,173.520001,215.230265,166.838368,157.853665
7,-41.966646,-45.096337,28.433743,-42.243844,-85.494532,20.578227,-35.422493,26.858452,-69.248987,-6.262041,...,22.516552,47.387110,-19.210429,62.933595,39.852646,-20.063221,69.107979,63.612475,119.039121,70.678006
8,-28.725592,-20.994639,12.964035,-5.918352,-25.317758,16.035425,-37.002177,9.556427,-28.512536,6.650666,...,18.386693,-21.108357,-14.476060,30.993046,-1.640568,0.609824,30.069405,40.030882,37.147241,27.989091
9,-81.301436,-57.537742,37.237299,-22.245534,-53.664566,27.220539,-80.985820,16.391046,-42.928244,20.782713,...,27.257518,-29.899684,-20.688406,84.828160,-7.967166,-5.421147,57.592650,94.121376,73.524405,60.605660


In [106]:
vector_phrase_title_news_skipgram = word2Vec_skipgram(phrase_title_news)
df_phrase_title_news_skipgram = pd.DataFrame(vector_phrase_title_news_skipgram)
df_phrase_title_news_skipgram

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.538541,-1.20618,0.476054,-0.733671,-0.598655,0.814311,-0.017775,-0.172323,-1.003629,-0.094528,...,-0.13664,-0.646587,-0.012262,-0.702797,-0.143575,0.111026,1.453186,0.634915,1.92312,0.610639


In [107]:
calculate_similiarity(df_full_news_skipgram,df_phrase_title_news_skipgram)

[0.5983094198753126,
 0.7047331703552803,
 0.5950589507227511,
 0.6989760221069802,
 0.5982522814633964,
 0.5636207756750163,
 0.5962808654963507,
 0.6730302832976581,
 0.6980378824315541,
 0.6148957848931662,
 0.6897142111254425,
 0.7231390029734502,
 0.49215290372735665,
 0.5846234312586817,
 0.686040800351223,
 0.7383783843359426,
 0.5897152898714844,
 0.811640228198912,
 0.7359962926537565,
 0.4967479986194572]

##Casos de testes 3 - Modelo Transformers

In [108]:
df_full_news_transformers = Vetorizacao(df_test_vectors['data'], vetorizadorTransformers)

In [109]:
vector_phrase_title_news_transformers = vetorizadorTransformers.encode(phrase_title_news)

In [110]:
calculate_similiarity(df_full_news_transformers, vector_phrase_title_news_transformers)

[0.6098183432356751,
 0.5838459640392386,
 0.5024042916985526,
 0.3577417822497904,
 0.5529091527285358,
 0.5526485985585573,
 0.39665428386150475,
 0.4051884650915668,
 0.6946555495295557,
 0.6551786760808287,
 0.5310021539751916,
 0.5584576012554496,
 0.3444487377025238,
 0.6587270342103078,
 0.6462994168998996,
 0.530390477829548,
 0.47332959985743905,
 0.5838459640392386,
 0.5485046860209941,
 0.6551786760808287]

##Casos de testes 4 - Modelo Bert

In [111]:
full_news_bert = Vetorizacao(df_test_vectors['data'], vetorizadorBert)
full_news_bert

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.037571,-0.134126,0.583761,0.112654,0.204770,0.178488,0.142073,-0.173217,0.089179,0.331715,...,-0.109005,0.191583,-0.304709,-0.552582,0.179850,-0.083643,0.274027,-0.064540,0.006353,-0.208390
1,0.047287,-0.406079,0.696057,0.107339,0.348362,0.124650,-0.212150,-0.176422,0.112529,0.155711,...,-0.054580,0.140933,-0.343495,-0.732228,0.260813,-0.148524,0.218278,0.023073,-0.179050,-0.039204
2,0.006295,-0.038772,0.472723,0.002407,0.060892,0.180560,0.001610,-0.019124,-0.017071,0.334687,...,0.009681,0.337262,-0.259232,-0.398253,0.049896,0.188736,0.151854,-0.137689,0.103421,0.031288
3,-0.052546,-0.290894,0.408679,0.023871,-0.245646,0.011187,-0.039897,0.014946,0.070618,0.346659,...,0.124947,0.177349,-0.449140,-0.142837,0.035818,0.094336,0.133570,0.056862,0.003069,-0.225844
4,0.091002,-0.275092,0.614563,0.142003,0.248706,0.218308,0.196526,-0.148189,0.021342,0.237373,...,-0.168077,0.146152,-0.229417,-0.617866,0.204632,-0.057292,0.241719,0.012886,0.073412,-0.137627
5,0.166029,-0.240711,0.633223,0.132679,0.073813,-0.055811,0.184122,-0.011314,0.001667,0.309815,...,-0.097076,0.310721,-0.240146,-0.292015,0.214166,0.072483,0.145336,0.041870,-0.065844,-0.008681
6,0.019097,-0.130792,0.686319,0.080190,0.135623,-0.164267,-0.059922,-0.006718,0.060772,0.351402,...,-0.065376,0.511272,-0.223821,-0.381608,0.141807,-0.115576,0.055948,-0.056580,-0.022582,0.008352
7,0.232904,-0.139818,0.706019,0.031965,0.108839,-0.047334,-0.105564,-0.174127,-0.157285,0.270049,...,-0.097537,0.411786,-0.396843,-0.440783,0.175776,-0.176976,0.194601,-0.120256,0.055828,0.040554
8,0.183132,-0.193198,0.529429,0.123516,0.272666,0.113360,0.029198,-0.103609,-0.079157,0.274650,...,-0.248267,0.360554,-0.170745,-0.566018,0.254850,-0.228539,0.162526,0.078295,-0.052866,0.060981
9,0.123641,-0.236952,0.565839,0.150594,0.332262,0.056304,-0.028292,-0.132892,-0.021003,0.351959,...,-0.263379,0.351623,-0.135058,-0.561452,0.219605,-0.247362,0.230648,0.017534,-0.023004,0.038001


In [112]:
vector_phrase_title_news_bert = vetorizadorBert.encode(phrase_title_news)

In [113]:
uau = calculate_similiarity(full_news_bert,vector_phrase_title_news_bert)
uau

[0.8892396420941024,
 0.8604003615332432,
 0.7509976689654063,
 0.7392201868711018,
 0.8831189225426556,
 0.7639971385515424,
 0.7998905664839204,
 0.8219511740203091,
 0.8698042532536748,
 0.8611014838243256,
 0.7983057702922689,
 0.9041129619915185,
 0.7652339896349853,
 0.8937684866973116,
 0.7917529889038404,
 0.9030714215506771,
 0.8856265632448094,
 0.8604003615332432,
 0.8630456206854807,
 0.8611014838243256]

#16.0 Testes das funções com resultado esperado

Aqui, conduzimos testes abrangentes em todas as funções, garantindo que os resultados estejam em conformidade com as expectativas estabelecidas para cada uma delas. É crucial destacar que esses testes abrangem todas as funcionalidades até a seção de Técnicas de Processamento de Linguagem Natural (PLN). No entanto, vale ressaltar que as funções de vetorização e similaridade do cosseno apresentam uma complexidade que torna difícil prever antecipadamente os resultados esperados. Devido a essa complexidade, nossa abordagem se concentra em realizar testes unitários exclusivamente para as funcionalidades de vetorização e similaridade do cosseno, garantindo assim um rigoroso controle de qualidade em relação a essas partes específicas do sistema.

In [126]:
import unittest

class TestStringManipulationFunctions(unittest.TestCase):
    def test_remocaoAcentos(self):
        result = remocaoAcentos("Café")
        self.assertEqual(result, "Cafe")

    def test_lowercase_text(self):
        result = lowercase_text("Hello World")
        self.assertEqual(result, "hello world")

    def test_clean_title(self):
        result = clean_title("Title - Subtitle")
        self.assertEqual(result, "Title")

    def test_remove_marks(self):
        result = remove_marks("Hello, World! This is a test.")
        self.assertEqual(result, "Hello World This is a test")

    def test_remove_aspas_series(self):
        input_data = pd.Series(['"Text 1"', '"Text 2"'])
        result = remove_aspas(input_data)
        expected_result = pd.Series(['Text 1', 'Text 2'])
        self.assertTrue(result.equals(expected_result))

    def test_remove_aspas_string(self):
        input_string = '"This is a string with quotes"'
        result = remove_aspas(input_string)
        expected_result = 'This is a string with quotes'
        self.assertEqual(result, expected_result)

    def test_remove_aspas_other_types(self):
        result = remove_aspas(12345)
        self.assertEqual(result, 12345)

    def test_remove_duplicate_spaces(self):
        result = remove_duplicate_spaces("   Hello   World   ")
        self.assertEqual(result, "Hello World")

    def test_remove_isolated_numbers(self):
        result = remove_isolated_numbers("Text 123 45 6 with numbers")
        self.assertEqual(result, "Text    with numbers")

    def test_extrair_titulo_com_sucesso(self):
        url = "https://www.google.com/"
        resultado = extrair_titulo(url)
        self.assertEqual(resultado, "Google")

    def test_remove_tags_com_sucesso(self):
        url = "https://www.google.com/"
        resultado = remove_tags(url)
        result_compare = 'google ai advertising business how search works our third decade of climate action join us privacy terms settings search settings advanced search your data in search search history search help send feedback dark theme off google apps'
        self.assertEqual(resultado, result_compare)

    def setUp(self):
        self.sample_data = [
            {
                "tag_removed": "Título da Notícia 12/12/2012",
                "titulo": "Título da Notícia"
            }
        ]

    def test_extract_dates(self):
        for data in self.sample_data:
            result = extract_dates(data)
            self.assertIsNotNone(result, "12/12/2012")

    def test_extrair_dominio(self):
        url = "https://exemplo.com/noticia"
        result = extrair_dominio(url)
        self.assertEqual(result, "exemplo.com")

    def test_named_entities_filtred(self):
        coluna = ["Apple foi fundada em 1976."]
        filtro = "ORG"
        result = named_entities_filtred(coluna, filtro)
        self.assertEqual(result, [{'Apple'}])

    def test_named_entities(self):
        coluna = ["Apple foi fundada em 1976."]
        result = named_entities(coluna)
        self.assertEqual(result, [[('Apple', 'ORG')]])

    def test_stopWords(self):
        dados = ["Este é um exemplo de texto com palavras de parada."]
        result = stopWords(dados)
        self.assertEqual(result, ['texto palavras parada .'])

    def test_lematizar(self):
        dados = ["Eu tenho corrido e eles correm."]
        result = lematizar(dados)
        self.assertEqual(result, ['eu ter corrido e eles correr .'])

    def test_tokenizar_texto(self):
        texto = "Isso é um exemplo."
        result = tokenizar_texto(texto)
        self.assertEqual(result, ["Isso", "é", "um", "exemplo", "."])


test_loader = unittest.TestLoader().loadTestsFromTestCase(TestStringManipulationFunctions)

test_runner = unittest.TextTestRunner(verbosity=2)
test_runner.run(test_loader)

test_clean_title (__main__.TestStringManipulationFunctions) ... ok
test_extract_dates (__main__.TestStringManipulationFunctions) ... ok
test_extrair_dominio (__main__.TestStringManipulationFunctions) ... ok
test_extrair_titulo_com_sucesso (__main__.TestStringManipulationFunctions) ... ok
test_lematizar (__main__.TestStringManipulationFunctions) ... ok
test_lowercase_text (__main__.TestStringManipulationFunctions) ... ok
test_named_entities (__main__.TestStringManipulationFunctions) ... ok
test_named_entities_filtred (__main__.TestStringManipulationFunctions) ... 

https://www.google.com/


ok
test_remocaoAcentos (__main__.TestStringManipulationFunctions) ... ok
test_remove_aspas_other_types (__main__.TestStringManipulationFunctions) ... ok
test_remove_aspas_series (__main__.TestStringManipulationFunctions) ... ok
test_remove_aspas_string (__main__.TestStringManipulationFunctions) ... ok
test_remove_duplicate_spaces (__main__.TestStringManipulationFunctions) ... ok
test_remove_isolated_numbers (__main__.TestStringManipulationFunctions) ... ok
test_remove_marks (__main__.TestStringManipulationFunctions) ... ok
test_remove_tags_com_sucesso (__main__.TestStringManipulationFunctions) ... ok
test_stopWords (__main__.TestStringManipulationFunctions) ... 

https://www.google.com/


ok
test_tokenizar_texto (__main__.TestStringManipulationFunctions) ... ok

----------------------------------------------------------------------
Ran 18 tests in 0.472s

OK


<unittest.runner.TextTestResult run=18 errors=0 failures=0>

#17.0 Funções auxiliares para a API

A fim de garantir que possamos recuperar mais de uma notícia em situações em que a similaridade é extraordinariamente alta, foi necessário implementar duas funções auxiliares em nossa API. Essas funções desempenham papéis essenciais na seleção e apresentação de notícias relevantes para o usuário.

1.   Retorna os index´s das noticias com maiores similaridades identificadas
2.   Retorna o conteúdo das noticias, buscando em nosso dataframe

Essas funções trabalham em conjunto para garantir que o usuário receba uma seleção das notícias mais relevantes e semelhantes ao input, melhorando assim a experiência de busca e recuperação de informações em nosso sistema.

##17.1 Retorna os indexs das noticias com maiores similaridades identificadas

Essa função tem a responsabilidade de identificar as notícias mais semelhantes ao input, limitando a saída a um máximo de três notícias. Uma vez localizadas, ela retorna os índices correspondentes dessas notícias no DataFrame que armazena nosso conjunto de notícias.

In [115]:
def encontrar_maiores(array):
    maiores = []
    for i, numero in enumerate(array):
        if numero > 0.85:
            maiores.append([numero, i])

    maiores.sort(reverse=True)

    if len(maiores) >= 3:
        return maiores[:3]
    elif len(maiores) > 0:
        return maiores
    else:
        return 'Nao ha nenhuma noticia correspondente em nosso banco de dados.'

##17.2 Retorna o conteúdo das noticias, buscando em nosso dataframe

Essa função, por sua vez, utiliza os índices previamente obtidos pela função anterior. Seu propósito é recuperar o conteúdo completo dessas notícias e organizá-lo em um formato JSON para apresentação ao usuário.

In [116]:
import json

def encontrar_noticias(indices, dataframe, coluna):
    noticias = []

    if str(type(indices)) == "<class 'str'>":
      return 'Nao ha nenhuma noticia correspondente em nosso banco de dados.'

    for i in indices:
        noticias.append(dataframe[coluna][i[1]])

    noticias_dict = {
        "noticia1": noticias[0] if len(noticias) > 0 else None,
        "noticia2": noticias[1] if len(noticias) > 1 else None,
        "noticia3": noticias[2] if len(noticias) > 2 else None
    }

    json_data = json.dumps(noticias_dict)

    return json_data

# 18.0 Processamento do database oficial em nossas pipelines e funções

Esse Dataframe é obtido por meio de uma requisição GET ao nosso back-end, que possui conexão com o banco de dados. Este dataframe será utilizado para recuperar as notícias compatíveis com o usuário em nossa API.

In [117]:
url = 'http://10.254.18.13:3001/v1/url/getall'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print('Erro ao obter os dados da API')

df_news_database = pd.DataFrame(data)

ConnectTimeout: ignored

In [ ]:
df_news_database

In [ ]:
df_news_database_vectors = pipeline_dataframe(df_news_database, "kkkk")
df_news_database_vectors

In [ ]:
df_full_news_transformers = Vetorizacao(df_news_database_vectors['kkkk'], vetorizadorTransformers)

# 19.0 Flask API

In [ ]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!pip install flask-bootstrap
!ngrok authtoken '2UlDYDSwwAVqompW0e1LnB6CW6f_7rmKmfbpYLmEP26GP3WW2'

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

# Rota de saúde
@app.route("/health", methods=['GET'])
def health_check():
    return "API está rodando sem erros", 200

# Rota para processar a entrada de voz
@app.route("/process_audio", methods=['POST'])
def process_audio():
    try:
      data = request.get_json()
      if data is not None and 'text' in data:
        text_value = data['text']

        text_value = pipeline_speech_to_text(text_value)

        vector_text = vetorizadorTransformers.encode(text_value)

        news_response = calculate_similiarity(df_full_news_transformers,vector_text)

        news_index = encontrar_maiores(news_response)

        news_response = encontrar_noticias(news_index, df_news, 'all_news')
        return news_response, 200
      else:
        return jsonify({"erro": "Dados JSON inválidos"}), 400
    except Exception as e:
        return jsonify({"erro": str(e)}), 500

@app.route("/call_external_api", methods=['GET'])
def update_news():
    url = 'http://10.254.18.13:3001/v1/url/getall'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
    else:
        print('Erro ao obter os dados da API')

    df_news_database = pd.DataFrame(data)

    df_news['all_news'] = df_news_database['url'].apply(remove_tags)

    df_news_database_vectors = pipeline_dataframe(df_news_database, "url")

    df_full_news_transformers = Vetorizacao(df_news_database_vectors['data'], vetorizadorTransformers)

    return df_full_news_transformers, 200

if __name__ == '__main__':
    app.run()